In [ ]:
library(data.table)
library(Seurat)
library(ggplot2)
library(grid)
library(dplyr)
library(pheatmap)
# library(DoubletFinder)
library(ggpubr)
library(TxDb.Hsapiens.UCSC.hg19.knownGene)
library(org.Hs.eg.db)
# library(BSgenome.Hsapiens.UCSC.hg19)
library(robustbase)
library(gridExtra)
library(RColorBrewer)
library(stringr)
library(MAST)
library(tibble)
library(fgsea)
library(ggpubr)
library(GEOquery)
library(AUCell)
library(stringr)
library(circlize)
library(ComplexHeatmap)
# library(monocle3)
library(fgsea)
library(stringr)
library(extraDistr)
library(entropy)
# library(writexl)
library(doRNG)
library(doParallel)

# source("../src/motif_analysis.R")
source("../src/process_pdac.R")
source("../src/pdac_plots.R")
base_path <- file.path("..","data")

s.genes <- c("MCM5","PCNA","TYMS","FEN1","MCM2","MCM4","RRM1","UNG","GINS2","MCM6","CDCA7","DTL","PRIM1","UHRF1","MLF1IP","HELLS","RFC2","RPA2","NASP","RAD51AP1","GMNN","WDR76","SLBP","CCNE2","UBR7","POLD3","MSH2","ATAD2","RAD51","RRM2","CDC45","CDC6","EXO1","TIPIN","DSCC1","BLM","CASP8AP2","USP1","CLSPN","POLA1","CHAF1B","BRIP1","E2F8")
g2m.genes <- c("HMGB2","CDK1","NUSAP1","UBE2C","BIRC5","TPX2","TOP2A","NDC80","CKS2","NUF2","CKS1B","MKI67","TMPO","CENPF","TACC3","FAM64A","SMC4","CCNB2","CKAP2L","CKAP2","AURKB","BUB1","KIF11","ANP32E","TUBB4B","GTSE1","KIF20B","HJURP","CDCA3","HN1","CDC20","TTK","CDC25C","KIF2C","RANGAP1","NCAPD2","DLGAP5","CDCA2","CDCA8","ECT2","KIF23","HMMR","AURKA","PSRC1","ANLN","LBR","CKAP5","CENPE","CTCF","NEK2","G2E3","GAS2L3","CBX5","CENPA")


The cell below loads the cell annotations and creates a Seurat object from the publicly available read count matrix. 

Two files --- cell_doublets.tsv and edge_info_all_doublet_filtered_celltypes.rds --- which contain (a) the list of doublet cells to be removed from the Seurat object, and (b) the edge/non-edge annotations for each cell. 

In [ ]:
base_path <- file.path("..","data")
pdac_matrix_file_name <- "CRA001160.matrix"
pdac_anno_file_name <- "CRA001160.celltypes.tsv"
pdac_matrix_path <- file.path( base_path, pdac_matrix_file_name )
pdac_annotation_path <- file.path( base_path, pdac_anno_file_name )

pdac_anno_dt <- process_annotations( pdac_annotation_path )
seurat_obj <- create_full_seurat_object( read_gene_exp_mat( 
    pdac_matrix_path ), pdac_anno_dt )
meta_data_dt <- data.table( seurat_obj@meta.data, keep.rownames = T ) %>% setnames(.,"rn","cell.name")

doublet_cells <- fread( file.path( base_path, "cell_doublets.tsv" ),header=F) %>% .$V1

filtered_cells <- meta_data_dt[!cell.name %in% doublet_cells,cell.name]
pdac_anno_dt <- pdac_anno_dt[cell.name %in% filtered_cells,]

seurat_obj <- subset( seurat_obj, cells=filtered_cells )
seurat_obj <- NormalizeData( seurat_obj )
meta_data_dt <- data.table( seurat_obj@meta.data, keep.rownames = T ) %>% setnames(.,"rn","cell.name")

edge_info_all <- readRDS( file.path( base_path, "edge_info_all_doublet_filtered_celltypes.rds" ) )



Run edge pipeline on all non-malignant cell types

In [ ]:
all_cell_types <- unique(pdac_anno_dt$cluster)
data_gene_exp_dt <- data.table(gene_name=rownames(seurat_obj))
lognorm_gene_exp_dt <- data.table(gene_name=rownames(seurat_obj))
data_mat <- seurat_obj[["RNA"]]@data
for (cell_type in all_cell_types) {
    if (cell_type == "Ductal cell type 2")
        next
    print(cell_type)
    flush.console()
    if (cell_type == "Acinar cell") {
        for (cell_category in c("edge","center")) {
            cells <- edge_info_all$edge_center_dt %>% dplyr::filter(cell_category == cell_category &
                                                                   normal_cell_type == "Acinar cell") %>% 
            pull(cell.name)
            data_gene_exp_dt <- count_gene_exp_dt %>% mutate(!!sym(paste(cell_category,"Acinar cell")):=
            rowMeans(data_mat[,cells]))
        }
    } else {
        cells <- WhichCells(seurat_obj,expression = cluster == cell_type)
        data_gene_exp_dt <- count_gene_exp_dt %>% mutate(!!sym(cell_type):=rowMeans(data_mat[,cells]))
    }
}
fwrite(data_gene_exp_dt,"Profiles_For_Deconvolution.tsv.gz",sep="\t")

In [ ]:
all_cell_types <- unique(pdac_anno_dt$cluster)
across_refs_edge_center_dt <- data.table()
num_pcs = 50

source("../src/process_pdac.R")
malignant_cell_types <- c("Ductal cell type 2")
normal_cell_types <- setdiff( unique(pdac_anno_dt$cluster), malignant_cell_types)
edge_info_all <- add_edge_center_annotation_classic( seurat_obj, malignant_cell_types=malignant_cell_types, 
                                  normal_cell_types=normal_cell_types, num_pcs=num_pcs, 
                                  perform_control=T, no_tumour_adjacent=F, sample_info_column="orig.ident" )

saveRDS(edge_info_all,"edge_info_all_doublet_filtered_celltypes.rds")


## Figure S1

This is a heatmap showing the distribution of the fraction of cells that each type that are contributed by each sample.

In [ ]:
seurat_obj <- FindVariableFeatures(seurat_obj) %>% ScaleData %>% RunPCA(npcs=50,verbose=F) %>% RunUMAP(dims=1:50,verbose=F)
fig_s1_umap <- DimPlot( seurat_obj, group.by="cluster", label=T ) + theme(legend.position="none")

sample_info_df <- seurat_obj@meta.data %>% dplyr::select(orig.ident,cluster)
per_sample_count_df <- group_by(sample_info_df,cluster,orig.ident) %>% dplyr::count() %>% ungroup  
total_cell_count_df <- group_by(per_sample_count_df,cluster) %>% summarize(n_total=sum(n)) %>%
merge(.,per_sample_count_df,by="cluster") %>% mutate(cell_frac=n/n_total) %>% 
dplyr::select(cluster,orig.ident,cell_frac,n)


total_cell_frac_mat <- tidyr::pivot_wider(total_cell_count_df %>% dplyr::select(-n),names_from="orig.ident",
                                          values_from="cell_frac") %>% tibble::column_to_rownames("cluster") %>% as.matrix
total_cell_frac_mat <- total_cell_frac_mat[,c(paste0("N",1:11),paste0("T",1:24))]

options(repr.plot.width=6,repr.plot.height=12)
#Since the cell type fractions vary in a very short range, we need to log-scale them in order to get a usable
#heatmap for display.
max_frac <- max(apply(log(total_cell_frac_mat),1,function(x){return(max(na.omit(x)))}))
min_frac <- min(apply(log(total_cell_frac_mat),1,function(x){return(min(na.omit(x)))}))
colors <- colorRamp2(c(min_frac,max_frac),c("#87ceeb","#ffa500"))
values <- c(-10,-8,-6,-4,-2,0)

p_heatmap <- Heatmap(log(total_cell_frac_mat),cluster_rows=F,cluster_columns=F,name="fraction",col=colors,
                     na_col="black",
       rect_gp = gpar(col = "black", lwd = 2),
    top_annotation = HeatmapAnnotation(
        Fraction = 1:35,
        annotation_legend_param = list(col_fun = colors, title = "Fraction", at = values,
           labels=formatC(exp(values),format="e"))))

fig_S1_umap_heatmap <- grid.arrange(
    arrangeGrob(fig_s1_umap,
    grid.grabExpr(draw(p_heatmap)),
     widths=c(1,2),
    ncol=2
))
ggsave("Fig-supplement-umap.svg",fig_S1_umap_heatmap,height=6,width=24)

## Figures 1,C,D

These are UMAP plots of acinar cells colored by edge/non-edge labels (Figure 1C) and by sample of origin (Figure 1D). Given the large number of samples from which the cells come from, we don't display the sample labels. 

In [ ]:
umap_plots <- list()
umap_plots2 <- list()
names_vec <- c("Acinar cell"="Acinar","Ductal cell type 1"="Ductal")
for (cell_type in c("Acinar cell","Ductal cell type 1")) {
        subset_obj <- subset( seurat_obj, cells=WhichCells(seurat_obj,expression=cluster == cell_type) )
    subset_obj <- AddMetaData(subset_obj,
                             merge(tibble::rownames_to_column(subset_obj@meta.data,"cell.name"),
                                  edge_info_all$edge_center_dt[,.(cell.name,cell_category)],by="cell.name") %>%
                             tibble::column_to_rownames("cell.name"))

    subset_obj <- FindVariableFeatures(subset_obj,nfeatures=1000) %>% ScaleData %>% RunPCA(.,npcs=50) %>%
    RunUMAP(.,dims=1:50)
    umap_plots[[cell_type]] <- DimPlot( subset_obj, group.by="cell_category" ) + 
    theme_pubr(base_size=13.33) + theme(legend.position="none") + ggtitle(names_vec[cell_type])
    if (cell_type == "Acinar cell") {
        umap_plots2[[cell_type]] <- DimPlot( subset_obj, group.by="orig.ident" ) + 
    theme_pubr(base_size=13.33) + ggtitle(names_vec[cell_type]) + theme(legend.position="none")
    }

}
rm(subset_obj)
ggsave( "Fig1-UMAPs.svg", ggarrange( plotlist=umap_plots, nrow=1, ncol=2, common.legend=F ), 
       width=5,height=2.5)
ggsave( "Fig1-UMAPs-by-sample.svg", ggarrange( plotlist=umap_plots2, nrow=1, ncol=1, common.legend=F ), 
       width=2.5,height=2.5)

## Figure 1A 

Example density plots of heterogeneity and proximity ratio tests

In [ ]:
skewness_dt <- compute_skewness( edge_info_all$control_dist_dt[cell_type %in% c("Acinar cell","Ductal cell type 1"),] )
edge_distance_dt <- compute_edge_distance_significance( edge_info_all$edge_malignant_dist_dt[normal_cell_type %in% c("Acinar cell","Ductal cell type 1"),] )

p_skew <- ggplot( skewness_dt[column_type != "actual",] ) + 
geom_density(aes(x=skewness,color=cell_type),linetype="dashed") + 
geom_vline(data=skewness_dt[column_type == "actual",],aes(xintercept=skewness,color=cell_type)) +
theme_pubr(base_size=13.33) + xlab("Skewness (s)") + ylab("Density") + theme(legend.position="none")
p_edge <- ggplot( edge_distance_dt[dist_type != "actual",] ) + 
geom_density(aes(x=edge_center_dist_ratio,color=normal_cell_type),linetype="dashed") + 
geom_vline(data=edge_distance_dt[dist_type == "actual",],aes(xintercept=edge_center_dist_ratio,color=normal_cell_type)) +
theme_pubr(base_size=13.33) + xlab("Proximity Ratio (R)") + ylab("Density") + theme(legend.position="none")

ggsave("Fig1B.svg",ggarrange(p_skew,p_edge,nrow=2,ncol=1),height=3.5,width=2.5)

## Derive marker genes for outlier populations in each cell type

Prior to marker gene derivation, cells are scored for cell cycle activity using the cell cycle genes provided on the Seurat website. The p-values for each marker gene in the edge vs non-edge comparison is computed by accounting for the covariates of cell cycle score, sample identity and the number of expressed genes.

Note that sample identity and the number of expressed genes do tend to highly co-vary, so omitting one of these two factors does not affect the p-values much.

In [ ]:
lr_markers_df_list <- list()
normal_cell_types <- setdiff(seurat_obj$cluster,c("Ductal cell type 2"))
for (cell_type in normal_cell_types) {
    subset_obj <- subset( seurat_obj, cells=WhichCells(seurat_obj,expression=cluster == cell_type) )
    subset_obj <- AddMetaData(subset_obj,
                             merge(tibble::rownames_to_column(subset_obj@meta.data,"cell.name"),
                                  edge_info_all$edge_center_dt[,.(cell.name,cell_category)],by="cell.name") %>%
                             tibble::column_to_rownames("cell.name"))

    subset_obj <- CellCycleScoring(subset_obj,s.genes,g2m.genes)
    lr_markers_df_list[[cell_type]] <- FindMarkers(subset_obj,ident.1="edge",ident.2="center",group.by="cell_category",
                                test.use="LR",latent.vars=c("nFeature_RNA","G2M.Score","S.Score","orig.ident")) %>%
    tibble::rownames_to_column("gene")
}

saveRDS(lr_markers_df_list,"all_outlier_markers.rds")

fwrite( lr_markers_df_list[["Acinar cell"]] %>% dplyr::filter(p_val_adj < 0.1 & avg_logFC > 0),
       "acinar_alt_edge_up_markers.tsv", sep="\t" )
fwrite( lr_markers_df_list[["Acinar cell"]] %>% dplyr::filter(p_val_adj < 0.1 & avg_logFC < 0),
       "acinar_alt_edge_down_markers.tsv", sep="\t" )
fwrite( lr_markers_df_list[["Ductal cell type 1"]] %>% dplyr::filter(p_val_adj < 0.1 & avg_logFC > 0),
       "ductal_alt_edge_up_markers.tsv", sep="\t" )
fwrite( lr_markers_df_list[["Ductal cell type 1"]] %>% dplyr::filter(p_val_adj < 0.1 & avg_logFC < 0),
       "ductal_alt_edge_down_markers.tsv", sep="\t" )

writexl::write_xlsx( lr_markers_df_list[["Acinar cell"]] %>% dplyr::filter(p_val_adj < 0.1) %>% 
mutate(gene_type=ifelse(avg_logFC < 0,"Edge-Down","Edge-Up")), "TableS3.xlsx" )

## Using pseudotime to define edge cells

Here, we explore how Monocle3's pseudotime values could be used to carry out the skewness and heterogeneity tests, instead of the Normal PC and Pooled PC-based calculations.


In [ ]:
subset_cells <- WhichCells(seurat_obj,expression=cluster %in% c("Acinar cell","Ductal cell type 2","Ductal cell type 1"))
subset_obj <- subset( seurat_obj, cells=subset_cells ) %>% NormalizeData %>% 
FindVariableFeatures 
s.genes <- c("MCM5","PCNA","TYMS","FEN1","MCM2","MCM4","RRM1","UNG","GINS2","MCM6","CDCA7","DTL","PRIM1","UHRF1","MLF1IP","HELLS","RFC2","RPA2","NASP","RAD51AP1","GMNN","WDR76","SLBP","CCNE2","UBR7","POLD3","MSH2","ATAD2","RAD51","RRM2","CDC45","CDC6","EXO1","TIPIN","DSCC1","BLM","CASP8AP2","USP1","CLSPN","POLA1","CHAF1B","BRIP1","E2F8")
g2m.genes <- c("HMGB2","CDK1","NUSAP1","UBE2C","BIRC5","TPX2","TOP2A","NDC80","CKS2","NUF2","CKS1B","MKI67","TMPO","CENPF","TACC3","FAM64A","SMC4","CCNB2","CKAP2L","CKAP2","AURKB","BUB1","KIF11","ANP32E","TUBB4B","GTSE1","KIF20B","HJURP","CDCA3","HN1","CDC20","TTK","CDC25C","KIF2C","RANGAP1","NCAPD2","DLGAP5","CDCA2","CDCA8","ECT2","KIF23","HMMR","AURKA","PSRC1","ANLN","LBR","CKAP5","CENPE","CTCF","NEK2","G2E3","GAS2L3","CBX5","CENPA")
subset_obj <- CellCycleScoring(subset_obj,s.genes,g2m.genes)

expr_mat <- subset_obj[["RNA"]]@data
all_genes <- rownames(expr_mat)
gene_meta_data <- data.frame( gene_short_name=all_genes, row.names=all_genes )

cds_obj <- new_cell_data_set( expr_mat, cell_metadata=subset_obj@meta.data %>% dplyr::rename(cell.type=cluster),
                             gene_metadata = gene_meta_data)
 

# flush.console()

### Pseudotime-based heterogeneity test

The pseudotime values of all cells are computed using the normal Monocle3 pipeline (with G2M.Score, S.Score, and batch information) regressed out. 

100 sets of shuffled cells are generated by permuting the PCA coordinates of cells amongst each other, and the skewness from running Monocle3 after each shuffle is computed.

In [ ]:
num_pca_shuffles <- 100
shuffle_random_seed <- 10
control_skewness_df <- data.frame()
pt_edge_cells_list <- list()
edge_cell_frac <- 0.1

cell_types <- c("Acinar cell","Ductal cell type 1")

for (normal_cell_type in cell_types) {
    subset_cells <- WhichCells(subset_obj,expression = cluster == normal_cell_type )
    subset_pseudotime_obj <- subset( subset_obj, cells=subset_cells ) %>% FindVariableFeatures
 
    skewness_df <- data.frame(cell_type=normal_cell_type,skewness=-1,skewness_type=-1,shuffle_num=0:num_pca_shuffles)
    skewness_vals <- c()
    skewness_type <- c()
    for (idx in 0:num_pca_shuffles) {
        if (idx == 0) {
            shuffle_pca <- F
            skewness_type <- c(skewness_type,"actual")
        } else {
            shuffle_pca <- T
            skewness_type <- c(skewness_type,"control")
        }
        subset_cds_obj <- compute_pseudotime( cds_obj[,subset_cells], VariableFeatures(subset_pseudotime_obj), 
                                         shuffle_pca=shuffle_pca, random.seed=idx )

        root_candidates <- WhichCells(subset_pseudotime_obj,expression = cluster == normal_cell_type )
        subset_cds_obj <- set_root(subset_cds_obj,root_candidates)
        umap_obj <- subset_cds_obj@principal_graph_aux$UMAP
        pseudotime_dt <- tibble::enframe(umap_obj$pseudotime,name="cell.name",value="pseudotime")
        
        if (idx == 0) {
            print(paste("Storing edge cells for",normal_cell_type))
            pt_edge_cells_list[[normal_cell_type]] <- pseudotime_dt %>% dplyr::filter(pseudotime > quantile(pseudotime,1-edge_cell_frac)) %>% pull(cell.name)
        }

        skewness_vals <- c(skewness_vals,mc(pseudotime_dt$pseudotime))
    }
    skewness_df$skewness <- skewness_vals
    skewness_df$skewness_type <- skewness_type
    control_skewness_df <- rbind(control_skewness_df,skewness_df)
}
fwrite(control_skewness_df,"trajectory_skewness_values.tsv",sep="\t")

Running the cell below outputs the p-values of the skewness test for acinar and ductal cells

In [ ]:
control_skewness_mat <- control_skewness_df %>% dplyr::filter(skewness_type == "control") %>% group_by(cell_type) %>%
summarize(m=mean(skewness),s=sd(skewness)) %>% tibble::column_to_rownames("cell_type") %>% as.matrix

skewness_p_values <- c()
for (cell_type_ in cell_types) {
    actual_skewness <- control_skewness_df %>% dplyr::filter(cell_type == cell_type_ & 
                                                               skewness_type == "actual") %>% pull(skewness)
     skewness_p_values <- c(skewness_p_values,1-pnorm(actual_skewness,
                                                    control_skewness_mat[cell_type_,"m"],
                                                    control_skewness_mat[cell_type_,"s"]))
}
names(skewness_p_values) <- cell_types
skewness_p_values

### Pseudotime-based proximity ratio test


In [ ]:
control_edge_distance_df <- data.frame()
num_label_shuffles <- 100
for (normal_cell_type in cell_types) {
    subset_cells <- WhichCells(subset_obj,expression = cluster %in% c(normal_cell_type,"Ductal cell type 2") )
    subset_pseudotime_obj <- subset( subset_obj, cells=subset_cells ) %>% FindVariableFeatures
    subset_cds_obj <- compute_pseudotime( cds_obj[,subset_cells], VariableFeatures(subset_pseudotime_obj), 
                                     shuffle_pca=F )

    root_candidates <- WhichCells(subset_pseudotime_obj,expression = cluster == normal_cell_type )
    subset_cds_obj <- set_root(subset_cds_obj,root_candidates)
    umap_obj <- subset_cds_obj@principal_graph_aux$UMAP
    pseudotime_dt <- tibble::enframe(umap_obj$pseudotime,name="cell.name",value="pseudotime")

    merged_df <- merge( pseudotime_dt, meta_data_dt[,.(cell.name,cluster)], by="cell.name" )
    median_pseudotimes <- merged_df %>% group_by(cluster) %>% 
    summarize(median_pseudotime=median(pseudotime)) %>% ungroup %>% tibble::deframe(.)
    normal_df <- merged_df %>% dplyr::filter(cluster==normal_cell_type) %>%
    mutate(cell_category=ifelse(cell.name %in% pt_edge_cells_list[[normal_cell_type]],"edge","center"))
    malignant_df <- merged_df %>% dplyr::filter(cluster!=normal_cell_type)


    ratio_vals <- c()
    edge_distance_df <- data.frame(cell_type=normal_cell_type,distance_type="control",
                                   edge_distance_ratio=-1,shuffle_num=0:num_label_shuffles)
    distance_types <- c()
    for (label_shuffle in 0:num_label_shuffles) {
        if (label_shuffle == 0) {
            ratio <- compute_pseudotime_distance_ratio(normal_df,malignant_df)
            distance_types <- c(distance_types,"actual")
        } else {
            normal_df_copy <- copy(normal_df) %>% mutate(cell_category=sample(cell_category))
            ratio <- compute_pseudotime_distance_ratio(normal_df_copy,malignant_df)
            distance_types <- c(distance_types,"control")
        }
        ratio_vals <- c(ratio_vals,ratio)
    }
    edge_distance_df$edge_distance_ratio <- ratio_vals
    edge_distance_df$distance_type <- distance_types
    control_edge_distance_df <- rbind(control_edge_distance_df,edge_distance_df)
}
fwrite(control_edge_distance_df,"trajectory_edge_distance_values.tsv",sep="\t")
control_edge_distance_mat <- control_edge_distance_df %>% dplyr::filter(distance_type == "control") %>% group_by(cell_type) %>%
summarize(m=mean(edge_distance_ratio),s=sd(edge_distance_ratio)) %>% tibble::column_to_rownames("cell_type") %>% as.matrix

edge_distance_p_values <- c()
for (cell_type_ in c("Acinar cell","Ductal cell type 1")) {
    actual_edge_distance <- control_edge_distance_df %>% dplyr::filter(cell_type == cell_type_ & 
                                                             distance_type == "actual") %>% pull(edge_distance_ratio)
    edge_distance_p_values <- c(edge_distance_p_values,pnorm(actual_edge_distance,
                                                    control_edge_distance_mat[cell_type_,"m"],
                                                    control_edge_distance_mat[cell_type_,"s"]))
}
names(edge_distance_p_values) <- cell_types
edge_distance_p_values

### Figure S2D

The null distributions of heterogeneity and proximity ratio tests for acinar and ductal cells based on pseudotime values.

In [ ]:
p_pseudotime_ratio <- ggplot( control_edge_distance_df %>% dplyr::filter(distance_type == "control") ) + 
geom_density(aes(x=edge_distance_ratio,color=cell_type),linetype="dashed") + 
geom_vline(data = control_edge_distance_df %>% dplyr::filter(distance_type == "actual"),
           aes(xintercept=edge_distance_ratio,color=cell_type) ) + xlab("Proximity Ratio") +
ylab("Density") + theme_pubr(base_size=10) + theme(legend.position="bottom")

p_pseudotime_skew <- ggplot( control_skewness_df %>% dplyr::filter(skewness_type == "control") ) + 
geom_density(aes(x=skewness,color=cell_type),linetype="dashed") + 
geom_vline(data = control_skewness_df %>% dplyr::filter(skewness_type == "actual"),
           aes(xintercept=skewness,color=cell_type) ) + xlab("Skewness") +
ylab("Density") + theme_pubr(base_size=10) + theme(legend.position="bottom")

ggsave( "supplementary-trajectory-edge-analysis.svg", 
       ggarrange( plotlist=list(p_pseudotime_skew, p_pseudotime_ratio), nrow=1,ncol=2),width=5,height=2.5 )

## Figure S1B

This is a check on the expression of markers genes in acinar, ductal and malignant cells

In [ ]:
seurat_obj <- SetIdent( seurat_obj, value="cluster")
acinar_malignant_obj <- subset( seurat_obj, idents=c("Acinar cell","Ductal cell type 2"
                                                    )) %>% NormalizeData

#The first four markers are malignant ductal markers, while the latter four are acinar markers.
markers <- c("KRT19","KRT7","TSPAN8","SLPI","PRSS1", "CTRB1", "CTRB2", "REG1B")
marker_dt <- data.table( FetchData( acinar_malignant_obj, vars=c(markers,"cluster")), keep.rownames=T ) %>% 
                        setnames(.,"rn","cell.name")
rm(acinar_malignant_obj)

options(repr.plot.width=12,repr.plot.height=7)
temp_df <- merge( marker_dt, edge_info_all$edge_center_dt[,.(cell.name,cell_category)], by="cell.name", all.x=T ) %>%
mutate(cell_category=ifelse(is.na(cell_category),"malignant",cell_category)) %>%
data.table %>% melt(id.vars=c("cell.name","cluster","cell_category"))
p <- ggviolin( temp_df, x="cell_category", y="value", fill="cell_category") + 
stat_compare_means(comparisons=list(c("edge","center"),c("malignant","edge")),aes(label=..p.signif..),size=5) + 
theme_pubr(base_size=12) + facet_wrap(~variable,nrow=2) + ylim(0,10) + scale_fill_manual(values=c("edge"="orange",
                                                                                            "center"="blue",
                                                                                            "malignant"="red")) +
scale_x_discrete(labels=c("center"="Non-Edge","edge"="Edge","malignant"="Malignant")) + xlab(NULL) + ylab("Norm. expression") +
theme(axis.text.x=element_text(angle=90,hjust=1,vjust=0.5),legend.position="none")# +
ggsave("supplement-marker-expression.svg",p,height=5,width=10)


## Figure 1B

Skewness and heterogeneity test plots for all cell types

Violin plot of the null distributions corresponding to the heterogeneity and proximity ratio tests (in gray). The actual heterogeneity and proximity ratio values are plotted as a point which is colored as blue/red depending on whether the p-values are significant.

In [ ]:
skewness_dt <- compute_skewness(edge_info_all$control_dist_dt)
skewness_dt[column_type == "actual",p_adj:=p.adjust(p_value,method="bonferroni")]
skewness_dt[column_type == "actual",significant:=fifelse(p_adj <= 0.05,TRUE,FALSE)]

p_skew <- ggplot( skewness_dt[column_type == "control",] ) + geom_violin(aes(x=cell_type,y=skewness),
                                                                         color="gray",fill="gray",width=0.5) + 
theme_pubr(base_size=5) + theme(axis.text.x=element_text(angle=90,hjust=1,vjust=0.5),
                                panel.grid.major=element_line(color="gray",size=0.1),
                                legend.position="none",
                               axis.line=element_line(size=0.2)) + 
ylab("Skewness") + xlab(NULL) + 
geom_point(data=skewness_dt[column_type == "actual",],
           aes(x=cell_type,y=skewness,color=significant),size=0.5 )

edge_distance_dt <- compute_edge_distance_significance(edge_info_all$edge_malignant_dist_dt)
edge_distance_dt[dist_type == "actual",p_adj:=p.adjust(p_value,method="bonferroni")]
edge_distance_dt[dist_type == "actual",significant:=fifelse(p_adj <= 0.05,TRUE,FALSE)]

p_distance <- ggplot( edge_distance_dt[dist_type == "control",] ) + geom_violin(aes(x=normal_cell_type,y=edge_center_dist_ratio),
                                                                  color="gray",fill="gray") + 
theme_pubr(base_size=5) + theme(axis.text.x=element_text(angle=90,hjust=1,vjust=0.5),
                                panel.grid.major=element_line(color="gray",size=0.1),
                                legend.position="none",
                               axis.line=element_line(size=0.2)) + 
ylab("Edge distance ratio") + xlab(NULL) + 
geom_point(data=edge_distance_dt[dist_type == "actual",],
           aes(x=normal_cell_type,edge_center_dist_ratio,color=significant),size=0.5)

p_final <- ggarrange(plotlist=list(p_skew,p_distance),ncol=1)
ggsave("fig1-fingerprints.svg",p_final,width=2,height=3)

## Figure S1C

Sub-sampling reads and genes from edge cells to check if the differences in library sizes between edge and non-edge cells is the sole reason for the appearance of edge acinar cells in skewness and heterogeneity tests.

In [ ]:
norm_type <- "Acinar cell"

options(repr.plot.width=8, repr.plot.height=3)
num_reps <- 100
for (resampling_type in c("reads","genes")) {
    skewness_p_values <- c()
    edge_distance_p_values <- c()
    for (rep in 1:num_reps){
        print(rep)
        flush.console()
        set.seed(rep)
        resample_info_list <- resample_edge_cells( seurat_obj, pdac_anno_dt, edge_info_all$edge_center_dt, 
                                                  c("Acinar cell"), c("Ductal cell type 2"), resampling_type,
                                              no_tumour_adjacent=F)
        ret <- plot_skewness(resample_info_list$edge_info$control_dist_dt)
        skewness_p_value <- ret$skewness_dt[1,p_value]
        ret <- plot_edge_distance_ratio(resample_info_list$edge_info$edge_malignant_dist_dt)
        edge_distance_p_value <- ret$edge_distance_dt[1,p_value]
        edge_distance_p_values <- c(edge_distance_p_values,edge_distance_p_value)
        skewness_p_values <- c(skewness_p_values,skewness_p_value)
    }
    saveRDS(list("edge-distance"=edge_distance_p_values,"skewness"=skewness_p_values),
            paste(resampling_type,"resampling.rds",sep="_"))
}

dt_list <- list()
idx <- 1
for (resampling_type in c("reads","genes")) {
    resampling_info <- readRDS(paste(resampling_type,"resampling.rds",sep="_"))
    for (test_type in c("skewness","edge-distance")) {
        dt_list[[idx]] <- data.table(p_value=resampling_info[[test_type]],resample_type=resampling_type,
                  test_type=test_type)
        idx <- idx + 1
    }
}
resample_info_dt <- rbindlist(dt_list)

p <- ggplot( resample_info_dt ) + geom_boxplot(aes(x=test_type,y=log10(p_value),color=resample_type),width=0.5) + 
geom_hline(yintercept = -1,color="red",linetype="dashed") +
theme_pubr(base_size=10) + xlab(NULL) + ylab("Log10(q-value)") + labs(color="Quantity resampled")
ggsave("supplement-resampling.svg",p,width=2.5,height=2.5)

## Figure S1A

Justifying merging normal and tumour-adjacent acinar cells. We carry out three tests (corresponding to the three panels in Figure S1A) to check if tumor-adjacent cells have a propensity to be more edge-like in their behaviour. 

In [ ]:
norm_type <- "Acinar cell"
tumour_adjacent_cells <- meta_data_dt[cluster == norm_type & sample_type == "tumour",cell.name]
num_pcs <- 50

acinar_edge_info <- add_edge_center_annotation_classic( seurat_obj, 
                                                       malignant_cell_types=c("Ductal cell type 2"), 
                                   normal_cell_types=c(norm_type), num_pcs=num_pcs, 
                                   perform_control=T, no_tumour_adjacent=T )

normal_acinar_obj <- subset( seurat_obj, subset = sample_type == "normal" & cluster == "Acinar cell")
normal_acinar_obj <- AddMetaData( normal_acinar_obj, 
                                tibble::column_to_rownames(acinar_edge_info$edge_center_dt[,.(cell.name,cell_category)],
                                                          "cell.name")) %>% SetIdent(value="cell_category") %>%
CellCycleScoring(.,s.features=s.genes,g2m.features=g2m.genes)

normal_only_edge_markers_df <- FindMarkers(normal_acinar_obj,ident.1="edge",test.use="LR",
                                          latent.vars=c("G2M.Score","S.Score","nFeature_RNA")) %>%
tibble::rownames_to_column("gene")
acinar_edge_cells <- WhichCells(normal_acinar_obj,idents="edge")
non_acinar_edge_cells <- WhichCells(normal_acinar_obj,idents="center")
rm(normal_acinar_obj)


#This displays the proximity ratio and heterogeneity test p-values after tumor-adjacent cells are excluded.
plot_edge_distance_ratio(acinar_edge_info$edge_malignant_dist_dt,plot_facets=T)$plot_obj
plot_skewness(acinar_edge_info$control_dist_dt,plot_facets=T)$plot_obj

In [ ]:
options(repr.plot.width=8, repr.plot.height=3)
theme_set(theme_classic())

merged_diff_exp_dt <- merge( lr_markers_df_list[["Acinar cell"]], 
                            normal_only_edge_markers_df,
                            by="gene",
                           suffixes=c(".adj_vs_center",".edge_vs_center")) %>% data.table
p1 <- ggplot( merged_diff_exp_dt ) + geom_point(aes(x=avg_logFC.edge_vs_center,y=avg_logFC.adj_vs_center)) + 
geom_line(aes(x=avg_logFC.edge_vs_center,y=avg_logFC.edge_vs_center),color="gray") + theme_pubr(base_size=8) +
geom_hline(aes(yintercept=0)) + geom_vline(aes(xintercept=0)) + xlab("Average Log FC\n(Edge vs Non-Edge)") +
ylab("Average Log FC\n(Tumor-adjacent vs Non-Edge)")
print(paste("R^2"=cor.test(merged_diff_exp_dt$avg_logFC.edge_vs_center,merged_diff_exp_dt$avg_logFC.adj_vs_center)))

plot_dt <- melt( merged_diff_exp_dt[,.(gene,avg_logFC.edge_vs_center,avg_logFC.adj_vs_center)],
                id.vars="gene", value.name="Average Log FC")
p2 <- ggplot( plot_dt ) + #geom_point(aes(x=variable,y=`Average Log FC`,color=variable),position="jitter") +
geom_boxplot(aes(x=variable,y=`Average Log FC`),fill=NA,width=0.4) + theme_pubr(base_size=8) +
scale_x_discrete("",labels=c("avg_logFC.edge_vs_center" = "Edge vs\n Non-Edge",
                          "avg_logFC.adj_vs_center" = "Tumor-Adjacent vs\n Non-Edge")) + 
stat_compare_means(aes(x=variable,y=`Average Log FC`),label.x=1.5,label="p.signif",hjust=0.5)

acinar_edge_center_dt <- edge_info_all$edge_center_dt[normal_cell_type == "Acinar cell",.(cell.name,dist_from_normal_medoid,dist_from_malignant_medoid)]
acinar_edge_center_dt <- merge( acinar_edge_center_dt, meta_data_dt[,.(cell.name,sample_type)] )
melted_dt <- melt(acinar_edge_center_dt[sample_type=="tumour",][,!c("sample_type")],id.vars=c("cell.name"))
p3 <- ggplot( melted_dt ) + stat_compare_means(aes(x=variable,y=value),label="..p.signif..",
                                               method.args=list("alternative"="g"),label.x=1.5,hjust=0.5) +
 geom_boxplot(aes(x=variable,y=value)) + theme_pubr(base_size=8) + scale_x_discrete(c("variable"=""), labels=c("dist_from_normal_medoid"="From\nacinar medoid",
                                                           "dist_from_malignant_medoid"="From\nmalignant medoid")) + ylab("Distance in PC space")
p <- ggarrange(p1,p2,p3,ncol=3,nrow=1)
ggsave("supplement-tumor-adjacent.svg",p,width=6,height=2)

### Evaluate relative proportions of cell cycle phases amongst edge and non-edge acinar cells

In [ ]:
acinar_obj <- subset( seurat_obj, subset=cluster=="Acinar cell") %>% CellCycleScoring(s.features=s.genes,g2m.features=g2m.genes)
acinar_obj <- AddMetaData( acinar_obj, 
                                tibble::column_to_rownames(edge_info_all$edge_center_dt[,.(cell.name,cell_category)],
                                                          "cell.name")) %>% SetIdent(value="cell_category")


acinar_obj@meta.data %>% dplyr::select(cell_category,Phase) %>% group_by(cell_category,Phase) %>% dplyr::count()
acinar_obj@meta.data %>% group_by(cell_category) %>% dplyr::count()

In [ ]:
acinar_obj@meta.data %>% dplyr::select(cell_category,Phase) %>% group_by(cell_category,Phase) %>% dplyr::count()
acinar_obj@meta.data %>% group_by(cell_category) %>% dplyr::count()

In [ ]:
#Proportions test for G1 phase
binom.test(610,1657,82/184 ) #p=0.36

In [ ]:
#Proportions test for G2 phase
binom.test(548,1657,72/184) #p=0.33

In [ ]:
#Proportions test for S phase
binom.test(499,1657,30/184) #p = 0.30

## Figure S2E

In [ ]:
n_total_dt <- edge_info_all_merged[normal_cell_type == "Acinar cell",] %>%
group_by(sample) %>% dplyr::count()
n_edge_dt <- edge_info_all_merged[normal_cell_type == "Acinar cell" & cell_category=="edge",] %>%
group_by(sample) %>% dplyr::count()
sample_edge_info_dt <- merge( n_edge_dt, n_total_dt, by="sample", suffixes=c("_edge","_total"), all.y=T ) %>%
tidyr::replace_na(list("n_edge"=0))

num_edge_cells <- sum(sample_edge_info_dt$n_edge)
num_draws <- 10000

#We draw N edge cells at random from all the acinar cells, and compute the fraction of acinar cells that
#come from each sample. We then compute the entropy of the distribution of fractions for a given draw.
#We carry out 10,000 draws, and compute a distribution of entropies, and compare it with the entropy
#of the fraction distribution of actual edge cells.
random_draws_mat <- rmvhyper( num_draws, sample_edge_info_dt$n_total, num_edge_cells )
random_draws_mat <- apply(random_draws_mat,2,function(x){return(x/num_edge_cells)})
colnames(random_draws_mat) <- sample_edge_info_dt$sample

random_entropy_vec <- apply(random_draws_mat,1,entropy)
actual_entropy <- entropy(sample_edge_info_dt$n_edge/num_edge_cells)
m <- mean(random_entropy_vec)
s <- sd(random_entropy_vec)

print("p-value of entropy of observed distribution of edge cell fractions from samples")
pnorm( actual_entropy, m, s)


In [ ]:
p_entropy <- ggplot( data.table(x=random_entropy_vec) ) + geom_density(aes(x=x)) + theme_pubr(base_size=13.33) +
xlab("Entropy of random draws") + ylab("Density")
ggsave("Fig-S2-entropy.svg",p_entropy,width=6,height=3)

## Finding doublets in reference PDAC data

The code below is largely boiler-plate code for running the doubletFinder pipeline. We assume a much higher doublet rate here than necessary to err on the side of caution. 

In [ ]:
doublet_info_list <- list()
samples <- unique( meta_data_dt$sample )
for (sample_id in samples) {
    sample_seurat_obj <- subset( seurat_obj, cells=meta_data_dt[sample == sample_id,cell.name])
    sample_seurat_obj <- NormalizeData(sample_seurat_obj) %>% FindVariableFeatures(.,nfeatures=1000) %>% 
    ScaleData(.) %>% RunPCA(.,npcs=50,verbose=F)  %>% RunUMAP(.,dims=1:50,verbose=F)
    
    homotypic.prop <- modelHomotypic(sample_seurat_obj@meta.data$cluster)
    nExp_poi <- round(0.05*nrow(sample_seurat_obj@meta.data))    #This is the expected doublet rate that we fix given ~3,000 cells per extraction.
    
    sample_param_sweep <- paramSweep_v3(sample_seurat_obj, PCs = 1:50, sct = FALSE)
    sample_param_sweep_summary <- summarizeSweep(sample_param_sweep, GT = FALSE)
    sweep_dt <- data.table( find.pK( sample_param_sweep_summary ) )
    optimal_pK <- as.double(as.vector(sweep_dt[order(-BCmetric),][1]$pK))
    sample_seurat_obj <- doubletFinder_v3(sample_seurat_obj, 
                               PCs = 1:50, pN = 0.25, pK = optimal_pK, nExp = nExp_poi, 
                                          reuse.pANN = FALSE, sct = FALSE)
    
    sample_meta_data_dt <- data.table( sample_seurat_obj@meta.data, keep.rownames = T ) %>% setnames(.,"rn","cell.name")
    doublet_class_col <- paste("DF.classifications_0.25",optimal_pK,nExp_poi,sep="_")
    doublet_info_list[[sample_id]] <- sample_meta_data_dt[,c("cell.name",doublet_class_col),with=F] %>%  setnames(.,doublet_class_col,"doublet_class")
}

merged_doublet_dt <- rbindlist(doublet_info_list)
cell_doublets <- meta_data_dt[cell.name %in% merged_doublet_dt[doublet_class == "Doublet",cell.name],cell.name]
fwrite( data.table(cell.name=cell_doublets), "cell_doublets.tsv", sep="\t", col.names=F, quote=F)

## TCGA methylation and RNA-seq

In [ ]:
library(ggplot2)
library(ggpubr)
ggTest <- list( c("EdgeUp", "RestOfGenome"), c("EdgeDown", "RestOfGenome"))

edge_acinar_markers_df <- lr_markers_df_list[["Acinar cell"]] %>% dplyr::filter(p_val_adj < 0.1)
edge_acinar_up_genes <- edge_acinar_markers_df %>% dplyr::filter(avg_logFC > 0) %>% pull(gene)
edge_acinar_down_genes <- edge_acinar_markers_df %>% dplyr::filter(avg_logFC < 0) %>% pull(gene)

outlier_ductal_markers_df <- lr_markers_df_list[["Ductal cell type 1"]] %>% dplyr::filter(p_val_adj < 0.1)
outlier_ductal_up_genes <- outlier_ductal_markers_df %>% dplyr::filter(avg_logFC > 0) %>% pull(gene)
outlier_ductal_down_genes <- outlier_ductal_markers_df %>% dplyr::filter(avg_logFC < 0) %>% pull(gene)


########################Expression@##########################
medianTumorZscores <- read.table("../data/MedianZscoreOfTumorSamplesForEdgeCellGenes.txt", sep = "\t", header = T)
medianTumorZscores <- medianTumorZscores %>% mutate(GeneType="RestOfGenome") %>%
mutate(GeneType=ifelse(Gene %in% edge_acinar_up_genes & CellType == "Acinar","EdgeUp",
                       GeneType)) %>%
mutate(GeneType=ifelse(Gene %in% edge_acinar_down_genes & CellType == "Acinar","EdgeDown",
                       GeneType)) %>%
mutate(GeneType=ifelse(Gene %in% outlier_ductal_up_genes & CellType == "Ductal","EdgeUp",
                       GeneType)) %>%
mutate(GeneType=ifelse(Gene %in% outlier_ductal_down_genes & CellType == "Ductal","EdgeDown",
                       GeneType))

medianTumorZscores$GeneType <- factor(medianTumorZscores$GeneType, levels = c("EdgeUp", "RestOfGenome", "EdgeDown"))
maxLimit <- boxplot.stats(medianTumorZscores$Zscore)$stats[c(1, 5)]	

bpG <- ggboxplot(na.omit(medianTumorZscores), 
				x="GeneType", y="Zscore",
				ylab=("Tumor Z-score"),
				color = "GeneType", lwd = 0.5, outlier.shape = NA, facet.by = "CellType") +
                theme_pubr() +
			theme(axis.text.y = element_text(size = 10),
				axis.title.x = element_blank(),
				axis.title.y = element_text(size = 10),
                 legend.position="none") + 
stat_compare_means(comparisons = ggTest, label = "p.format", size = 3)

# ggsave( "TumorZscoresForEdgeCellGenesFromGtex.svg", width = 5, height = 3, pointsize = 10, units = 'in' )


##############################Methylation################################
methylationDf <- fread("../data/TumorAndNoralSamplesMethylationForEdgeCellGenes.txt", sep = "\t", header = T) %>%
dplyr::rename(Gene=GeneName)

methylationDf <- methylationDf %>% mutate(GeneType="RestOfGenome") %>%
mutate(GeneType=ifelse(Gene %in% edge_acinar_up_genes & CellType == "Acinar","EdgeUp",
                       GeneType)) %>%
mutate(GeneType=ifelse(Gene %in% edge_acinar_down_genes & CellType == "Acinar","EdgeDown",
                       GeneType)) %>%
mutate(GeneType=ifelse(Gene %in% outlier_ductal_up_genes & CellType == "Ductal","EdgeUp",
                       GeneType)) %>%
mutate(GeneType=ifelse(Gene %in% outlier_ductal_down_genes & CellType == "Ductal","EdgeDown",
                       GeneType))


methylationDf$GeneType <- factor(methylationDf$GeneType, levels = c("EdgeUp", "RestOfGenome", "EdgeDown"))



#############Plot delta##############
maxLimit <- boxplot.stats(methylationDf$deltaMethylation)$stats[c(1, 5)]	
bpG <- ggboxplot(na.omit(methylationDf), 
				x="GeneType", y="deltaMethylation",
				ylab=("Delta (Tumor - Normal)"),
				color = "GeneType", lwd = 0.5, outlier.shape = NA, facet.by = "CellType") + theme_pubr() +
			theme(axis.text.y = element_text(size = 10),
				axis.title.x = element_blank(),
				axis.title.y = element_text(size = 10),
                 legend.position="none") +
coord_cartesian(ylim = (maxLimit + c(-0.2, 0.3))) + 
stat_compare_means(method = "wilcox.test", comparisons = ggTest, label = "p.format", size = 3, label.y =
    c(0.3,0.35))

ggsave("MethylationDeltaAmongGeneCategoriesFromArrayExpress.svg", bpG, width = 5, height = 3, pointsize = 10, units = 'in')


In [ ]:
lr_markers_df_list <- readRDS("all_outlier_markers.rds")
outlier_up_list <- lapply(lr_markers_df_list,function(x){
    return(x %>% dplyr::filter(avg_logFC > 0 & p_val_adj < 0.1) %>% pull(gene))
})

outlier_down_list <- lapply(lr_markers_df_list,function(x){
    return(x %>% dplyr::filter(avg_logFC < 0 & p_val_adj < 0.1) %>% pull(gene))
})

seurat_obj <- NormalizeData( seurat_obj ) %>% ScaleData
scaled_mat <- seurat_obj[["RNA"]]@scale.data
clusters <- unique(seurat_obj$cluster)
mean_scaled_mat <- matrix(0,nrow=nrow(scaled_mat),ncol=length(unique(clusters)),
                         dimnames=list(rownames(scaled_mat),clusters))

for (cluster_ in clusters) {
    mean_scaled_mat[,cluster_] <- rowMeans(scaled_mat[,WhichCells(seurat_obj,expression = cluster == cluster_)])
}
rm(scaled_mat)

filtered_outlier_up_list <- list()
filtered_outlier_down_list <- list()
z_score_diff_thresh <- 0.1

for (cluster_ in clusters) {
    up_genes <- outlier_up_list[[cluster_]]
    down_genes <- outlier_down_list[[cluster_]]
    up_diff_vec <- vector(mode="logical",length=length(up_genes))  | TRUE
    down_diff_vec <- vector(mode="logical",length=length(down_genes)) | TRUE
    for (other_cluster in setdiff(clusters,cluster_)) {
        up_diff_vec <- up_diff_vec & mean_scaled_mat[up_genes,cluster_] - mean_scaled_mat[up_genes,other_cluster] > z_score_diff_thresh
        down_diff_vec <- down_diff_vec & mean_scaled_mat[down_genes,cluster_] - mean_scaled_mat[down_genes,other_cluster] > z_score_diff_thresh
    }
    filtered_outlier_up_list[[cluster_]] <- names(which(up_diff_vec == TRUE))
    filtered_outlier_down_list[[cluster_]] <- names(which(down_diff_vec == TRUE))

}

In [ ]:
z_score_diff_thresh <- 0.1

for (cluster_ in clusters) {
    up_genes <- outlier_up_list[[cluster_]]
    down_genes <- outlier_down_list[[cluster_]]
    up_diff_vec <- vector(mode="logical",length=length(up_genes))  | TRUE
    down_diff_vec <- vector(mode="logical",length=length(down_genes)) | TRUE
    for (other_cluster in setdiff(clusters,cluster_)) {
        up_diff_vec <- up_diff_vec & mean_scaled_mat[up_genes,cluster_] - mean_scaled_mat[up_genes,other_cluster] > z_score_diff_thresh
        down_diff_vec <- down_diff_vec & mean_scaled_mat[down_genes,cluster_] - mean_scaled_mat[down_genes,other_cluster] > z_score_diff_thresh
    }
    filtered_outlier_up_list[[cluster_]] <- names(which(up_diff_vec == TRUE))
    filtered_outlier_down_list[[cluster_]] <- names(which(down_diff_vec == TRUE))

}

In [ ]:
fwrite( tibble::enframe(filtered_outlier_up_list[["Ductal cell type 1"]],name="idx",value="gene"),
       "ductal_filtered_up_markers.tsv",sep="\t")
fwrite( tibble::enframe(filtered_outlier_down_list[["Ductal cell type 1"]],name="idx",value="gene"),
       "ductal_filtered_down_markers.tsv",sep="\t")

In [ ]:
outlier_up_list <- lapply(lr_markers_df_list,function(x){
    return(x %>% dplyr::filter(avg_logFC > 0 & p_val_adj < 0.1) %>% pull(gene))
})

outlier_down_list <- lapply(lr_markers_df_list,function(x){
    return(x %>% dplyr::filter(avg_logFC < 0 & p_val_adj < 0.1) %>% pull(gene))
})

ggTest <- list( c("Outlier_Up", "RestOfGenome"), c("Outlier_Down", "RestOfGenome"))
cell_type <- "Ductal cell type 1"
outlier_up_genes <- filtered_outlier_up_list[[cell_type]]#which(rowSums(exp_frac_mat[outlier_up_list[[cell_type]],other_types] > exp_thresh) <= 0) %>% names
outlier_down_genes <- filtered_outlier_down_list[[cell_type]]#which(rowSums(exp_frac_mat[outlier_down_list[[cell_type]],other_types] > exp_thresh) <= 0) %>% names

medianTumorZscores %>% dplyr::select(Gene,Zscore) %>% unique %>% mutate(GeneType="RestOfGenome") %>%
mutate(GeneType=ifelse(Gene %in% outlier_up_genes,"Outlier_Up",GeneType)) %>%
mutate(GeneType=ifelse(Gene %in% outlier_down_genes,"Outlier_Down",GeneType)) -> temp_df
temp_df$GeneType <- factor(temp_df$GeneType,levels=c("Outlier_Up","RestOfGenome","Outlier_Down"))

# temp_df %>% group_by(GeneType) %>% dplyr::count()
# ggplot( temp_df, aes(x=GeneType,y=Zscore) ) + geom_boxplot() + 
# theme_pubr(base_size=15) + stat_compare_means(comparisons=list(c("RestOfGenome","Outlier_Up"),
#                                                               c("RestOfGenome","Outlier_Down"),
#                                                               c("Outlier_Up","Outlier_Down"))) +
# ggtitle(paste(cell_type,"TCGA RNA-seq"))

bpG <- ggboxplot(na.omit(temp_df), 
				x="GeneType", y="Zscore",
				ylab=("Tumor Z-score"),
				color = "GeneType", lwd = 0.5, outlier.shape = NA) +
                theme_pubr() +
			theme(axis.text.y = element_text(size = 10),
				axis.title.x = element_blank(),
				axis.title.y = element_text(size = 10),
                 legend.position="none") + 
stat_compare_means(comparisons = ggTest, label = "p.format", size = 3)
ggsave( "TumorZscoresForEdgeCellGenesFromGtex_filtered.svg", width = 2.8, height = 2.7, pointsize = 10, units = 'in' )

methylationDf %>% dplyr::select(Gene,deltaMethylation) %>% unique %>% mutate(GeneType="RestOfGenome") %>%
mutate(GeneType=ifelse(Gene %in% outlier_up_genes,"Outlier_Up",GeneType)) %>%
mutate(GeneType=ifelse(Gene %in% outlier_down_genes,"Outlier_Down",GeneType)) -> temp_df
temp_df$GeneType <- factor(temp_df$GeneType,levels=c("Outlier_Up","RestOfGenome","Outlier_Down"))

bpG <- ggboxplot(na.omit(temp_df), 
				x="GeneType", y="deltaMethylation",
				ylab=("Delta (Tumor - Normal)"),
				color = "GeneType", lwd = 0.5, outlier.shape = NA) + theme_pubr() +
			theme(axis.text.y = element_text(size = 10),
				axis.title.x = element_blank(),
				axis.title.y = element_text(size = 10),
                 legend.position="none") +
coord_cartesian(ylim = (maxLimit + c(-0.2, 0.3))) + 
stat_compare_means(method = "wilcox.test", comparisons = ggTest, label = "p.format", size = 3, label.y =
    c(0.3,0.35))

ggsave("MethylationDeltaAmongGeneCategoriesFromArrayExpress_filtered.svg", bpG, width = 2.7, height = 2.8, pointsize = 10, units = 'in')

ggtitle(paste(cell_type,"TCGA methylation"))

## Comparison with markers of ADM and acinar dedifferentiation

We isolate two sets of marker genes, one for acinar-ductal metaplasia (ADM) and acinar de-differentiation. 
We subset out only these marker genes in the Seurat object for computing Log-FC values and their corresponding p-values. Note that we do not rely on adjusted p-values for determining which markers are significant, since we do not use these results for any downstream analyses.

### Figure 2A

In [ ]:
marker_table <- fread("../data/progenitor_marker_table.tsv")[,!c("Stainings\ and\ References")]
set(marker_table,NULL,"Acinar -> Progenitor",paste(marker_table$`Acinar cell`, marker_table$`Embryonic progenitor`,sep=" to "))
set(marker_table,NULL,"Ductal -> Progenitor",paste(marker_table$`Duct cell`, marker_table$`Embryonic progenitor`,sep=" to "))
marker_table <- marker_table[,!c("Acinar cell","Duct cell","Embryonic progenitor","")]
marker_genes <- marker_table$Marker

new_markers <- c("STAT3","SEL1L","CBL","KLF4","CTNND1","LKB1","ICAM1","DCLK1","CDKN1A") #Markers of ADM
marker_genes <- c(marker_genes,new_markers) %>% setdiff(.,c("KRT19","PARM1","GP2")) #These markers in Baldan et. al pertain to ductal cells and not acinar cells

sub_meta_data_dt <- meta_data_dt[cluster %in% c("Acinar cell"),.(cell.name,cluster)]
sub_meta_data_dt <- merge( sub_meta_data_dt, edge_info_all$edge_center_dt[,.(cell.name,cell_category)], by="cell.name")
sub_meta_data_dt[cluster == "Acinar cell",cell_type:=fifelse(cell_category == "edge","Edge Acinar","Non-Edge Acinar")]
sub_meta_data <- as.data.frame(sub_meta_data_dt[,list(cell_type)])
rownames(sub_meta_data) <- sub_meta_data_dt$cell.name
subset_seurat_obj <- subset( seurat_obj,
                            cells=sub_meta_data_dt$cell.name,
                            features=marker_genes)
subset_seurat_obj <- AddMetaData( subset_seurat_obj, sub_meta_data ) %>% SetIdent( ., value="cell_type")

#Setting min.pct = 0 and logfc.threshold = -Inf is the only way to force Seurat to compute LogFC values
#of all genes.
prog_markers_df <- FindMarkers( subset_seurat_obj, ident.1="Edge Acinar", min.pct = 0, logfc.threshold=-Inf) %>% 
tibble::rownames_to_column("gene") %>%  mutate(significant=ifelse(p_val < 0.01,TRUE,FALSE),
                                              regulation_type=ifelse(avg_logFC < 0,"Down","Up")) %>%
mutate(display_gene=ifelse(significant,paste0("*",gene),gene)) %>% merge( marker_table[,.(Marker,`Acinar -> Progenitor`)], by.x="gene", by.y="Marker",all.x=T)
for (gene in marker_genes) {
    if (!gene %in% marker_table$Marker){
        marker_table <- rbind( marker_table, list(gene,"- to +","- to +"))
    }
}

options(repr.plot.height=6,repr.plot.width=3)
prog_markers_df$gene <- factor(prog_markers_df$gene, marker_genes )
p <- ggplot( prog_markers_df ) + geom_bar(aes(x=gene,y=abs(avg_logFC),fill=regulation_type),
                                     stat="identity") + coord_flip() +
 geom_text(data = prog_markers_df %>% dplyr::filter(significant), aes(x=gene,y=abs(avg_logFC)),label="*", 
           hjust=-0.2, vjust=0.75, size=8) +
theme_pubr(base_size=10) + theme(legend.position="none",axis.text.x=element_text(size=10)) + 
scale_fill_manual(values=c("Down"="blue","Up"="red")) + xlab(NULL) + ylab("|Log-FC|") + ylim(0,4)
print(p)
ggsave("Fig2-markers.svg",p,width=3,height=6.5)
rm(subset_seurat_obj)

### Find marker genes of MPC and MPC-like cells in Vilani et. al

In [ ]:
if (!file.exists(file.path(base_path,"progenitor_annotation.tsv"))) {
    progenitor_seurat_obj <- process_progenitor_data()
 } else {
    progenitor_count_mat <- read_gene_exp_mat( file.path(base_path,"GSM4194789_TMM_counts_CPM.csv.gz"))
    progenitor_meta_data_dt <- fread(file.path(base_path,"progenitor_annotation.tsv"))
    progenitor_seurat_obj <- create_full_seurat_object( progenitor_count_mat, progenitor_meta_data_dt )
}

progenitor_seurat_obj <- SetIdent( progenitor_seurat_obj, value="seurat_clusters" )
marker_lists <- list("Multipotent"=c("SOX9","PTF1A","PDX1","NKX6-1"))

df <- FetchData( progenitor_seurat_obj, vars=unique(unlist(marker_lists)) )
mpc_cells <- df %>% dplyr::filter(SOX9 > 0 & PTF1A > 0 & PDX1 > 0 & `NKX6-1` > 0) %>% rownames

meta_data_df <- data.frame(cell.name=Cells(progenitor_seurat_obj),seurat_clusters=progenitor_seurat_obj$seurat_clusters) %>% 
mutate(seurat_clusters=ifelse(seurat_clusters == 5,"MPC-like",seurat_clusters)) %>%
mutate(seurat_clusters=ifelse(cell.name %in% mpc_cells,"MPC",seurat_clusters)) %>% 
tibble::column_to_rownames("cell.name")
progenitor_seurat_obj <- AddMetaData( progenitor_seurat_obj, meta_data_df ) %>% SetIdent(value="seurat_clusters")
progenitor_seurat_obj <- SetIdent( progenitor_seurat_obj, value="seurat_clusters" ) %>% NormalizeData
mpc_markers_df <- FindMarkers( progenitor_seurat_obj, 
                                          ident.1="MPC")
mpc_like_markers_df <- FindMarkers( progenitor_seurat_obj, 
                                       ident.1="MPC-like" )

### Figure S3B

Violin plots of multipotent and bipotent marker genes in each of the clusters in embryonic pancreas data.

In [ ]:
options(repr.plot.height=6,repr.plot.width=6)
plot_list <- VlnPlot( progenitor_seurat_obj, features=marker_lists[["Multipotent"]], 
        group.by="seurat_clusters",pt.size=0.1, combine=F )
plot_list <- lapply(plot_list,function(p){return(p+theme(legend.position="none",title=element_text(size=7)))})
p_markers <- ggarrange(plotlist=plot_list,nrow=2,ncol=2)
ggsave("progenitor_markers.svg",p_markers,width=6,height=4)

### Figure S3A

In [ ]:
progenitor_seurat_obj <- NormalizeData(progenitor_seurat_obj) %>% ScaleData(.) %>% 
FindVariableFeatures(.,nfeatures = 1000) %>% RunPCA(.,dims=50) %>% RunUMAP(.,dims=1:50)

options(repr.plot.height=6,repr.plot.width=6)
p_umap <- DimPlot( progenitor_seurat_obj, group.by="seurat_clusters",label=T,label.size=4) + 
theme(legend.position="none")
ggsave("progenitors_umap.svg",p_umap,width=6,height=6)

In [ ]:
DimPlot( progenitor_seurat_obj, group.by="seurat_clusters" )

In [ ]:
options(repr.plot.height=12,repr.plot.width=12)
FeaturePlot( progenitor_seurat_obj, features=c("SOX9","NKX6-1","PTF1A","PDX1"))

### Figure 2B

In [ ]:
#This tells us the number of cells in each cluster.
progenitor_seurat_obj@meta.data %>% group_by(seurat_clusters) %>% dplyr::count()

In [ ]:
mpc_like_genes <- mpc_like_markers_df %>% dplyr::filter(p_val_adj < 0.1 & avg_logFC > 0) %>% rownames
mpc_genes <- mpc_markers_df %>% dplyr::filter(p_val_adj < 0.1 & avg_logFC > 0) %>% rownames

gene_sets <- list("Multipotent"=mpc_genes,
                  "MPC-like"=mpc_like_genes)

acinar_and_ductal_cells <- meta_data_dt %>% dplyr::filter(cluster %in% c("Ductal cell type 1","Acinar cell")) %>% pull(cell.name)
acinar_mat <- seurat_obj[["RNA"]]@counts[,acinar_and_ductal_cells]
cells_rankings <- AUCell_buildRankings( acinar_mat, nCores=6, plotStats = F )
auc_obj <- AUCell_calcAUC( gene_sets, cells_rankings, nCores=6 )
auc_df <- getAUC(auc_obj) %>% t %>% as.data.frame %>% tibble::rownames_to_column("cell.name") %>% 
merge(.,edge_info_all$edge_center_dt[,.(cell.name,cell_category,normal_cell_type)],by="cell.name")

melted_auc_df <- auc_df %>% dplyr::select(-cell.name) %>% tidyr::pivot_longer(cols=c("Multipotent","MPC-like"),
                                                      values_to="AUC_score",names_to="Signature") %>% 
mutate( cell_category=ifelse(cell_category == "center","Non-Edge","Edge"))
melted_auc_df$cell_category <- factor(melted_auc_df$cell_category,levels=c("Edge","Non-Edge"))

options(repr.plot.width=4,repr.plot.height=3)
p <- ggviolin( melted_auc_df, x="cell_category",y="AUC_score",fill="cell_category",
                                    size=0.2,width=0.5,color="cell_category" ) + 
facet_wrap(vars(Signature,normal_cell_type),nrow=1) + theme_pubr(base_size=10) + 
xlab(NULL) + theme(panel.grid.major=element_line(color="gray",linetype="dotted"),legend.position="none") +
stat_compare_means(comparisons=list(c("Edge","Non-Edge")),
                   label.x=1.5,hjust=0.5,label="p.signif",
                   size=6,method.args=list("alternative"="g")) + ylim(0,0.5) + ylab("AUCell score")
p
ggsave("Fig2b.svg",p,width=5.5,height=2)

## Figure 2D

Below, we first compute an N x 3 matrix, where N = number of genes, and the columns represent the mean gene expression z-score of genes in acinar cells, normal ductal cells and malignant ductal cells. Note that the z-scoring is done after these three cell types are subsetted from the Seurat object. 

In [ ]:
subset_cells <- meta_data_dt %>% filter(cluster %in% c("Ductal cell type 1",
                                                       "Ductal cell type 2","Acinar cell")) %>% pull(cell.name)
subset_seurat_obj <- subset( seurat_obj, cells=subset_cells ) %>% ScaleData(.)

subset_meta_data_dt <- meta_data_dt[cell.name %in% subset_cells,.(cluster,cell.name)]
subset_meta_data_dt[,cell_ident:="Malignant"]
cluster_names <- list("edge Acinar cell"="Edge Acinar","center Acinar cell"="Non-Edge Acinar",
                     "edge Ductal cell type 1"="Outlier Ductal","center Ductal cell type 1"="Non-Outlier Ductal",
                     "Ductal cell type 2"="Malignant")
for (cell_type in c("Acinar cell","Ductal cell type 1")) {
    for (cell_category_ in c("edge","center")) {
        cells <- edge_info_all$edge_center_dt[normal_cell_type == cell_type & cell_category == cell_category_,cell.name]
        subset_meta_data_dt[cell.name %in% cells,cell_ident:=cluster_names[[paste(cell_category_,cell_type)]]]
    }
}
to_add_df <- as.data.frame(subset_meta_data_dt[,!c("cell.name")])
rownames(to_add_df) <- subset_meta_data_dt$cell.name
subset_seurat_obj <- AddMetaData(subset_seurat_obj,to_add_df) %>% SetIdent(.,value="cell_ident")

scaled_data <- subset_seurat_obj[["RNA"]]@scale.data
identities <- c("Edge Acinar","Non-Edge Acinar","Outlier Ductal","Non-Outlier Ductal","Malignant")
zscores_mat <- matrix(0,nrow=nrow(scaled_data),ncol=length(identities)+1,
                     dimnames=list(rownames(scaled_data),c(identities,"All Ductal")))
for (ident in identities) {
    zscores_mat[,ident] <- scaled_data[,WhichCells(subset_seurat_obj,ident=ident)] %>% rowMeans(.)
}
zscores_mat[,"All Ductal"] <- scaled_data[,subset_meta_data_dt[cluster == "Ductal cell type 1",cell.name]] %>% rowMeans(.)
rm(scaled_data)
rm(subset_seurat_obj)

In [ ]:
column_groups <- list("NEA-EA-M"=c("Non-Edge Acinar","Edge Acinar","Malignant"),
                     "NEA-EA-D1-M"=c("Non-Edge Acinar","Edge Acinar","All Ductal","Malignant"),
                     "NEA-EA-OD-M"=c("Non-Edge Acinar","Edge Acinar","Outlier Ductal","Malignant"),
                     "NEA-EA-NOD-M"=c("Non-Edge Acinar","Edge Acinar","Non-Outlier Ductal","Malignant"),
                     "NOD-OD-M"=c("Non-Outlier Ductal","Outlier Ductal","Malignant"))

path_mat_list <- list()
for (column_group in names(column_groups)) {
    path_mat <- zscores_mat[,column_groups[[column_group]]]
    path_mat_list[[column_group]] <- path_mat
}

enrichment_dt <- data.table()
threshold <- 0.2    #Threshold above which the difference in z-scores between two cell types is considered as significant.

#        in cluster | not in cluster
#   in set
#   not in set
for (column_group in names(path_mat_list)) {
    path_mat <- path_mat_list[[column_group]]
    differences <- t(apply(path_mat,1,diff))
    changing_genes_mask <- rowSums(differences > threshold) == ncol(differences)
    changing_genes <- rownames(path_mat)[changing_genes_mask]
    group_enrichment_dt <- compute_enrichment(changing_genes,rownames(seurat_obj))

    enrichment_dt <- rbind(enrichment_dt,group_enrichment_dt[,path:=column_group])
}
enrichment_dt[,p_adj:=p.adjust(p_value)]


## Figure S3C

In [ ]:
options(repr.plot.width=3,repr.plot.height=12)
pathway_mat <- tidyr::pivot_wider( enrichment_dt[,!c("p_value","q_value","odds_ratio")], names_from="path", values_from="p_adj" ) %>%
tibble::column_to_rownames("pathway") %>% as.matrix
pathway_mat <- pathway_mat[rowSums(pathway_mat < 0.1) >= 1,]
log_pathway_mat <- -log10(pathway_mat)
log_pathway_mat[log_pathway_mat < 1] <- NA
col_fun <- colorRamp2(c(0,40), c("blue","red"))
p_heatmap <- Heatmap(log_pathway_mat,cluster_columns=F,cluster_rows=F,col=col_fun,na_col="white",rect_gp = gpar(col="black"),
       column_names_gp=gpar(fontsize=8),row_names_gp=gpar(fontsize=8), width=unit(1,"in"), name="Log pvalue") #%>% as.ggplot


options(repr.plot.width=8,repr.plot.height=12)
figS2 <- grid.arrange(
    arrangeGrob(p_markers + labs(tag="A") + theme(plot.margin=unit(c(0,1,0,0),"cm"),plot.tag.position="topleft"),
                p_umap + labs(tag="B") + theme(plot.margin=unit(c(0,1,0,0),"cm"),plot.tag.position="topleft")),
    as.ggplot(p_heatmap) + labs(tag="C") + 
            theme(plot.tag.position="topleft",plot.margin=unit(c(0,0,0,-1.5),"cm")),
    heights=c(2,1),
    ncol=2
)
ggsave("Fig-S3-new.svg",figS2,width=8,height=12)

## Mechanisms of edge-ness

### Motif analysis

In [ ]:
atac_peaks_dt <- fread(file.path(base_path,"acinar_atac_peaks.narrowPeak"))[,.(chr=V1,start=V2,end=V3)]

all_genes <- rownames(seurat_obj[["RNA"]]@data)
entrez_dt <- get_entrez_dt( all_genes )
entrez_ids <- entrez_dt$final_entrez_id

entrez_dt[,gene_type:="Expressed"]
cds_tx_dt <- select(TxDb.Hsapiens.UCSC.hg19.knownGene,columns=c("GENEID","TXNAME"),
               keytype=c("GENEID"), keys=entrez_ids ) %>% na.omit(.)
cds_tx_dt <- merge( cds_tx_dt, entrez_dt[,.(SYMBOL,final_entrez_id,gene_type)],
                   by.x="GENEID", by.y="final_entrez_id") %>% data.table(.)

atac_peaks_dt <- gene_assign( atac_peaks_dt, assign_type="overlap", dist_threshold = 10000 )
           

### Peak-wise motif search

In [ ]:
source("../src/motif_analysis.R")

subset_motifs <- function(all_tfs,frac_threshold=0.1) {
    tf_names <- c()
    for (motif in all_tfs) {
        if (str_detect(motif,"::")) {
            motif <- str_split(motif,"::") %>% unlist
        }

        reformatted <- str_replace_all(motif,"\\(..*\\)","") %>% str_trim %>% str_to_upper
        tf_names <- c(tf_names,reformatted)
    }
    tf_names <- unique(tf_names)

    seurat_obj <- SetIdent( seurat_obj, value="cluster")
    acinar_seurat_obj <- subset( seurat_obj, idents="Acinar cell")
    tfs_present <- tf_names[tf_names %in% rownames(acinar_seurat_obj)]
    frac_expressed <- rowMeans(acinar_seurat_obj[["RNA"]][tfs_present,] > 0)
    tfs_present[frac_expressed >= frac_threshold]
    rm(acinar_seurat_obj)
    
    motifs_to_use <- list()
    for (tf in tfs_present) {
        motifs <- str_detect(all_tfs,tf)    
        motifs_to_use <- c(motifs_to_use,all_tfs[motifs])
    }
    motifs_to_use <- unique(motifs_to_use)
    return(motifs_to_use)
}

In [ ]:
motifs_to_use <- subset_motifs(names(merged_pwm_list))
promoter_atac_peaks_dt <- atac_peaks_dt[!is.na(SYMBOL),c("chr","start","end","SYMBOL","gene_type")] %>% unique#gene_assign( atac_peaks_dt, assign_type="overlap", dist_threshold = 10000 )
peaks_not_near_promoter_dt <- atac_peaks_dt[is.na(SYMBOL),c("chr","start","end","SYMBOL","gene_type")]
peaks_not_near_promoter_dt[,SYMBOL:=paste("seq",1:nrow(peaks_not_near_promoter_dt),sep="_")]

promoter_atac_granges <- makeGRangesFromDataFrame( promoter_atac_peaks_dt, keep.extra.columns=T ) %>% resize(.,width=100,fix="center")
atac_not_near_promoter_granges <- makeGRangesFromDataFrame( peaks_not_near_promoter_dt, keep.extra.columns=T ) %>% resize(.,width=100,fix="center")

pwm_list <- list()
for (motif in motifs_to_use) {
    pwm_list[[motif]] <- merged_pwm_list[[motif]]
}
edge_with_non_edge_list <- pwm_scan( pwm_list, 
                               foreground_granges_obj = promoter_atac_granges, 
                               background_granges_obj = atac_not_near_promoter_granges,
                               motif_p_value_thresh = 1e-5 )

enrichment_dt <- edge_with_non_edge_list$enrichment_dt %>% na.omit
tfs <- unique(enrichment_dt$tf)
foreground_dt <- edge_with_non_edge_list$fg_matches_dt
foreground_dt[,`:=`(gene_type=promoter_atac_peaks_dt$gene_type,SYMBOL=promoter_atac_peaks_dt$SYMBOL)]

idx <- 1

motif_gene_sets <- list()
for (tf_ in tfs) {
    print(idx)
    flush.console()
    tf_dt <- foreground_dt[get(tf_) == 1,c(tf_,"gene_type","SYMBOL"),with=F] %>% unique(.)
    motif_gene_sets[[tf_]] <- unique(tf_dt$SYMBOL)

   idx <- idx+1
}
enr_tfs <- enrichment_dt[q_value < 0.1,][order(num_fg),tf]

In [ ]:
saveRDS(motif_gene_sets,"Acinar_TF_Targets.rds")

In [ ]:
seurat_obj <- SetIdent(seurat_obj,value="cluster")
acinar_gene_exp_mat <- seurat_obj[["RNA"]]@counts[,WhichCells(seurat_obj,ident="Acinar cell")]
rankings <- AUCell_buildRankings( acinar_gene_exp_mat, nCores=6 )
rm(acinar_gene_exp_mat)
auc_obj <- AUCell_calcAUC( motif_gene_sets, rankings )
auc_mat <- getAUC(auc_obj)
auc_thresholds <- AUCell_exploreThresholds( auc_obj, plotHist=F, nCores=3 )
tfs <- names(auc_obj)
threshold_vec <- sapply(tfs,function(x){return(auc_thresholds[[x]]$aucThr$thresholds["Global_k1","threshold"]);})
binarized_auc_mat <- sapply(1:length(tfs),function(x){return(auc_mat[x,] > threshold_vec[x]);}) %>% t
rownames(binarized_auc_mat) <- tfs

num_bins <- 3
bin_column_names <- paste("Bin",1:num_bins)

acinar_edge_center_dt <- edge_info_all$edge_center_dt[normal_cell_type == "Acinar cell",]
acinar_edge_cells <- acinar_edge_center_dt[cell_category == "edge",cell.name]

distance_quantiles <- quantile(acinar_edge_center_dt$dist_from_normal_medoid, seq(0,1,by=1/num_bins))
num_quantiles <- length(distance_quantiles)
bin_cells_list <- list()


for (q_idx in 1:(num_quantiles-1)) {
    lower <- distance_quantiles[q_idx]
    upper <- distance_quantiles[q_idx+1]
    bin_cells_list[[q_idx]] <- acinar_edge_center_dt[dist_from_normal_medoid >= lower & 
                                                     dist_from_normal_medoid < upper,cell.name]
}

binned_auc_mat <- matrix(0,nrow=nrow(binarized_auc_mat),ncol=num_bins)
for (bin in 1:num_bins) {
    binned_auc_mat[,bin] <- rowMeans(binarized_auc_mat[,bin_cells_list[[bin]]])
}
rownames(binned_auc_mat) <- rownames(binarized_auc_mat)
colnames(binned_auc_mat) <- paste("Bin",1:num_bins)

gene_set_cv <- apply(binned_auc_mat,1,sd)/rowMeans(binned_auc_mat)
most_var_gene_sets <- names(sort(gene_set_cv,decreasing=T))[1:50]
options(repr.plot.width=6,repr.plot.height=12)
pheatmap(binned_auc_mat[c(most_var_gene_sets),],cluster_cols=F,fontsize=15,cutree_rows=2)

saveRDS(binned_auc_mat[c(most_var_gene_sets),],"binned_auc_mat_most_var_new.rds")
saveRDS(binned_auc_mat,"binned_auc_mat_new.rds")

writexl::write_xlsx( data.table( binned_auc_mat, keep.rownames=T) %>% setnames("rn","gene_name"),
                    "Table_S2.xlsx" )

## Figure 2C

Gene set activity changes from non-edge to edge cells

In [ ]:
acinar_seurat_obj <- subset( seurat_obj, subset = cluster == "Acinar cell") %>% ScaleData(.,features=rownames(seurat_obj))
num_bins <- 3
bin_column_names <- paste("Bin",1:num_bins)

acinar_edge_center_dt <- edge_info_all$edge_center_dt[normal_cell_type == "Acinar cell",]

distance_quantiles <- quantile(acinar_edge_center_dt$dist_from_normal_medoid, seq(0,1,by=1/num_bins))
num_quantiles <- length(distance_quantiles)
gene_exp_mat <- matrix(0,nrow=length(rownames(acinar_seurat_obj)),ncol=num_bins,
                      dimnames=list(rownames(acinar_seurat_obj),1:num_bins))

for (q_idx in 1:(num_quantiles-1)) {
    lower <- distance_quantiles[q_idx]
    upper <- distance_quantiles[q_idx+1]
    cells <- acinar_edge_center_dt[dist_from_normal_medoid >= lower & 
                                                     dist_from_normal_medoid < upper,cell.name]
    gene_exp_mat[,q_idx] <- rowMeans(acinar_seurat_obj[["RNA"]]@scale.data[,cells])
}
inter_bin_diff_mat <- apply(gene_exp_mat,1,diff) %>% t
z_score_thresh <- 0.1

top_num <- ceiling( 0.2 * length(rownames(seurat_obj )))
foreground_genes <- names(which(inter_bin_diff_mat[,1] > z_score_thresh & inter_bin_diff_mat[,2] > z_score_thresh))

#Ordinarily, we would select genes which are expressed in at least 1% of cells. However, there is such a 
#large change in gene expression from the non-edge to edge state that this filter effectively removes most of the
#genes expressed in Bin 1.
background_genes <- names(which(rowSums(acinar_seurat_obj[["RNA"]]@counts) > 0))

enrichment_dt <- compute_enrichment(foreground_genes,background_genes)

mascaux_gene_sets <- c("MYC_TARGETS_V2","MYC_TARGETS_V1","MTORC1_SIGNALING","PI3K_AKT_MTOR_SIGNALING",
                      "E2F_TARGETS","MITOTIC_SPINDLE","INTERFERON_GAMMA_RESPONSE","ALLOGRAFT_REJECTION",
                      "G2M_CHECKPOINT","IL6_JAK_STAT_SIGNALING","TNFA_SIGNALING_VIA_NFKB","INFLAMMATORY_RESPONSE",
                      "IL2_STAT5_SIGNALING","ADIPOGENESIS","EPITHELIAL_MESENCHYMAL_TRANSITION","UV_RESPONSE_UP","UV_RESPONSE_DOWN",
                      "OXIDATIVE_PHOSPHORYLATION","FATTY_ACID_METABOLISM","HYPOXIA")
enrichment_dt[q_value < 0.1 & pathway %in% paste("HALLMARK",mascaux_gene_sets,sep="_"),]
rm(acinar_seurat_obj)

writexl::write_xlsx( enrichment_dt[q_value < 0.1,.(pathway,odds_ratio,p_value,q_value)], 
                    "TableS1.xlsx" )

## Regression to find genes increasing/decreasing in expression with age in acinar cells in source PDAC data

In [ ]:
original_age_dt <- data.table( sample=c(paste0("T",1:24),paste0("N",1:11)),
                              age=c(64,52,58,72,65,64,70,66,36,61,51,54,58,67,54,56,71,
                                    68,59,59,59,67,54,44,64,55,50,53,52,31,42,41,34,65,30),
                             gender=c("M","M","F","F","F","M","M","F","M","M","M","M","F","F","F","F",
                                      "F","F","F","M","M","F","M","F", 
                                      "F","M","M","M","F","F","F","M","M","F","F"))

original_age_dt <- merge( edge_info_all$edge_center_dt[normal_cell_type == "Acinar cell",.(cell.name,cell_type=cell_category)],
                 meta_data_dt[,.(cell.name,sample)], by="cell.name" ) %>% merge(.,original_age_dt,by="sample")

original_acinar_obj <- subset( seurat_obj, subset=cluster == "Acinar cell") %>%
 ScaleData %>% CellCycleScoring(.,s.features = s.genes,g2m.features=g2m.genes)
original_age_dt <- merge( original_age_dt, 
                         tibble::rownames_to_column(original_acinar_obj@meta.data,"cell.name" ),
                         by="cell.name" )
output_df <- gene_exp_regression( original_acinar_obj[["RNA"]]@scale.data,
                                 original_age_dt %>% tibble::column_to_rownames("cell.name"),
                                 c("nFeature_RNA","S.Score","G2M.Score","age","gender"),c("age"))

age_increasing_genes <- output_df[age_q_value < 0.1 & age_coef > 0,gene]
num_cells_thresh <- ceiling(0.01 * length(Cells(original_acinar_obj)))
non_zero_expressed_genes <- names(which(rowSums(original_acinar_obj[["RNA"]]@counts > 0) > num_cells_thresh))

lr_markers_df_list <- readRDS("all_outlier_markers.rds")
all_edge_acinar_genes <- lr_markers_df_list[["Acinar cell"]] %>% dplyr::filter(avg_logFC > 0 & p_val_adj < 0.1) %>% pull(gene)
dt <- compute_enrichment( age_increasing_genes, non_zero_expressed_genes, 
                         pathways = list("Edge"=all_edge_acinar_genes))
                                        #"Multipotent"=multipotent_genes,
                                        #"Bipotent"=bipotent_genes))
print(dt)

all_edge_acinar_genes <- setdiff(all_edge_acinar_genes,age_increasing_genes)

fwrite(output_df[age_q_value < 0.1 & age_coef > 0,],"acinar_increasing_with_age_sex_regressed.tsv",sep="\t")
fwrite(output_df[age_q_value < 0.1 & age_coef < 0,],"acinar_decreasing_with_age_sex_regressed.tsv",sep="\t")
fwrite( lr_markers_df_list[["Acinar cell"]] %>% dplyr::filter(p_val_adj < 0.1) %>% 
mutate(gene_type=ifelse(avg_logFC > 0,"Upregulated","Downregulated")) %>% arrange(-avg_logFC),
      "acinar_edge_markers.tsv", sep="\t")

## Edge cells in other datasets

### GSE81547

In [ ]:
# detach("package:monocle3") #Monocle3 breaks the pData call from GEOquery. This hack circumvents the issue.
aging_obj <- process_GSE81547()
aging_meta_data_dt <- data.table( aging_obj@meta.data, keep.rownames=T ) %>% setnames(.,"rn","cell.name")
# library(monocle3)

In [ ]:
cel_seq_obj <- AddMetaData( cel_seq_obj, tibble::rownames_to_column(cel_seq_obj@meta.data,"cell.name") %>% 
                          merge(.,cel_seq_meta_data_dt[,.(cell.name,age)]) %>% tibble::column_to_rownames("cell.name"))
cel_seq_obj$sample_col <- paste("GSE85241",cel_seq_obj$age,sep="_")
aging_obj$sample_col <- paste("GSE81547",aging_obj$age,sep="_")
merged_obj <- merge( aging_obj[,aging_acinar_cells], cel_seq_obj[,vano_acinar_cells] )

In [ ]:
source("../src/process_pdac.R")
# aging_obj <- SetIdent(aging_obj,value="cell_type")
# aging_acinar_cells <- WhichCells(aging_obj,idents="acinar")
auc_info <- compute_AUCell_scores(merged_obj,gene_set)
shuffled_auc_info <- compute_shuffled_gene_set_AUCell_scores(merged_obj,gene_set,
                                          sample_info_column="sample_col")

In [ ]:
final_auc_df$log_nCount_RNA <- log10(final_auc_df$nCount_RNA)
temp <- glm( edge ~ 1 + log_nCount_RNA , data=final_auc_df )
final_auc_df$edge_residual <- temp$residuals

In [ ]:
fitted <- normalmixEM(temp$residuals,lambda=0.8)
final_auc_df$posterior_ratio <- log(fitted$posterior[,"comp.1"]) - log(fitted$posterior[,"comp.2"])
final_auc_df <- final_auc_df %>% mutate(is_edge=ifelse(posterior_ratio > 1,1,0))

In [ ]:
cel_seq_meta_data_dt$age %>% unique

In [ ]:
aging_obj$age %>% unique

In [ ]:
ggplot( final_auc_df ) + geom_point(aes(x=age,y=thresh,color=as.factor(age))) + facet_wrap(~age)

In [ ]:
thresholds_df <- shuffled_auc_info %>% group_by(sample) %>% summarize(thresh=max(mean))
final_auc_df <- tibble::rownames_to_column( as.data.frame(auc_info$auc_mat),
                            "cell.name" ) %>% merge(.,
                                                    tibble::rownames_to_column(merged_obj@meta.data,"cell.name")) %>%
merge(.,thresholds_df,by.x="sample_col",by.y="sample") %>% mutate(is_edge=ifelse(edge/thresh >= 1.5,1,0))

In [ ]:
fitted <- normalmixEM(final_auc_df$edge/final_auc_df$thresh,lambda=0.5)
final_auc_df$posterior_ratio <- log(fitted$posterior[,"comp.2"]) - log(fitted$posterior[,"comp.1"])
final_auc_df <- final_auc_df %>% mutate(is_edge=ifelse(posterior_ratio > 0.75,1,0))

In [ ]:
ggplot( final_auc_df %>% mutate(dataset=str_match(sample_col,"GSE[0-9][0-9]*") %>% unlist %>% as.character) ) + 
geom_density(aes(x=edge/thresh,color=dataset))

In [ ]:
ggplot( final_auc_df %>% group_by(age) %>% summarize(edge_frac=mean(is_edge)) ) +
geom_point(aes(x=age,y=edge_frac))

In [ ]:
final_auc_df

#### Computing the edge cell fraction

In [ ]:
aging_obj <- SetIdent(aging_obj,value="cell_type")
aging_acinar_cells <- WhichCells(aging_obj,idents="acinar")
aging_acinar_count_mat <- aging_obj[["RNA"]]@counts[,aging_acinar_cells]
gene_set <- list("edge"=all_edge_acinar_genes)
source("../src/process_pdac.R")
aucell_info <- compute_gene_set_AUCell_scores(aging_acinar_count_mat,gene_set,aging_meta_data_dt)
efsun_aucell_dt <- aucell_info$aucell_dt
efsun_info_tbl <- compute_edge_frac( efsun_aucell_dt )

#### Figure 4A : Prepare ADM/dedifferentiation marker plot

In [ ]:
sub_meta_data_dt <- aging_meta_data_dt[cell_type == "acinar",.(cell.name,cluster=cell_type,nFeature_RNA)]
sub_meta_data_dt <- merge( sub_meta_data_dt, 
                          efsun_aucell_dt, by="cell.name")

subset_seurat_obj <- subset( aging_obj,
                            cells=sub_meta_data_dt$cell.name ) %>%
CellCycleScoring(.,s.features=s.genes,g2m.features=g2m.genes) 

subset_seurat_obj <- AddMetaData( subset_seurat_obj, sub_meta_data_dt %>%dplyr::select(cell.name,age,nFeature_RNA,cell_type) %>%
                                tibble::column_to_rownames("cell.name")) %>% SetIdent( ., value="cell_type")
options(repr.plot.height=6,repr.plot.width=3)
efsun_markers_df <- FindMarkers(subset_seurat_obj,test.use="LR",ident.1="edge",
                                    latent.vars=c("S.Score","G2M.Score","nFeature_RNA"))
 
subset_seurat_obj <- subset_seurat_obj %>% subset(.,features=marker_genes)
prog_markers_df <- FindMarkers( subset_seurat_obj, ident.1="edge", min.pct = 0, logfc.threshold=-Inf) %>% 
tibble::rownames_to_column("gene") %>%  
mutate(significant=ifelse(p_val < 0.01,TRUE,FALSE),
                                               regulation_type=ifelse(avg_logFC < 0,"Down","Up")) %>%
mutate(display_gene=ifelse(significant,paste0("*",gene),gene)) %>% merge( marker_table[,.(Marker,`Acinar -> Progenitor`)], by.x="gene", by.y="Marker",all.x=T)
prog_markers_df$gene <- factor(prog_markers_df$gene, marker_genes )

p_efsun_markers <- ggplot( prog_markers_df ) + geom_bar(aes(x=gene,y=abs(avg_logFC),fill=regulation_type),
                                     stat="identity") + coord_flip() +
 geom_text(data = prog_markers_df %>% dplyr::filter(significant), aes(x=gene,y=abs(avg_logFC)),label="*", 
           hjust=0.0, vjust=0.75, size=8) +
theme_pubr(base_size=13.33) + theme(legend.position="none") + 
scale_fill_manual(values=c("Down"="blue","Up"="red")) + xlab(NULL) + ylab("|Log-FC|") + ylim(0,0.75)
rm(subset_seurat_obj)

### GSE85241

#### Computing the edge cell fraction

In [ ]:
out <- process_GSE85241()
cel_seq_obj <- out$seurat_obj
cel_seq_meta_data_dt <- out$meta_data
vano_acinar_cells <- out$acinar_cells
rm(out)

source("../src/process_pdac.R")
acinar_count_mat <- cel_seq_obj[["RNA"]]@counts[,vano_acinar_cells]
gene_set <- list("edge"=all_edge_acinar_genes)
aucell_info <- compute_gene_set_AUCell_scores( cel_seq_obj[["RNA"]]@counts[,vano_acinar_cells],gene_set,
                                             cel_seq_meta_data_dt )
vano_aucell_dt <- aucell_info$aucell_dt
vano_info_tbl <- compute_edge_frac( vano_aucell_dt )

#### Figure 4A : Prepare ADM/dedifferentiation marker plot

In [ ]:
sub_meta_data_dt <- cel_seq_meta_data_dt[cell.name %in% vano_acinar_cells,]#.(cell.name,cluster=cell_type)]
sub_meta_data_dt <- merge( sub_meta_data_dt, 
                          vano_aucell_dt[,.(cell.name,cell_type)], by="cell.name")

sub_meta_data <- as.data.frame(sub_meta_data_dt[,list(cell_type)])
rownames(sub_meta_data) <- sub_meta_data_dt$cell.name
subset_seurat_obj <- subset( cel_seq_obj,
                            cells=sub_meta_data_dt$cell.name ) %>% CellCycleScoring(.,
                                                                                   s.features=s.genes,
                                                                                   g2m.features=g2m.genes)
subset_seurat_obj <- AddMetaData( subset_seurat_obj, sub_meta_data ) %>% SetIdent( ., value="cell_type")
subset_seurat_obj <- subset( subset_seurat_obj, features=marker_genes)

prog_markers_df <- FindMarkers( subset_seurat_obj, ident.1="edge", min.pct = 0, logfc.threshold=-Inf) %>% 
tibble::rownames_to_column("gene") %>%  
mutate(significant=ifelse(p_val < 0.1,TRUE,FALSE),regulation_type=ifelse(avg_logFC < 0,"Down","Up")) %>%
mutate(display_gene=ifelse(significant,paste0("*",gene),gene)) %>% merge( marker_table[,.(Marker,`Acinar -> Progenitor`)], by.x="gene", by.y="Marker",all.x=T)
prog_markers_df$gene <- factor(prog_markers_df$gene, marker_genes )
p_vano_markers <- ggplot( prog_markers_df ) + geom_bar(aes(x=gene,y=abs(avg_logFC),fill=regulation_type),
                                     stat="identity") + coord_flip() +
 geom_text(data = prog_markers_df %>% dplyr::filter(significant), aes(x=gene,y=abs(avg_logFC)),label="*", 
           hjust=0.0, vjust=0.75, size=8) +
theme_pubr(base_size=13.33) + theme(legend.position="none") + 
scale_fill_manual(values=c("Down"="blue","Up"="red")) + xlab(NULL) + ylab("|Log-FC|") + ylim(0,2)

### Figure 4F

In [ ]:
gse81547_acinar_obj <- subset( aging_obj, subset=cell_type == "acinar") 
gse81547_acinar_obj <- AddMetaData( gse81547_acinar_obj, 
                                  efsun_aucell_dt[,.(cell.name,cell_type=paste0("GSE81547_",cell_type),
                                                    orig.ident="GSE81547")] %>%
                                  tibble::column_to_rownames("cell.name") ) %>% SetIdent(value="cell_type")

gse85241_acinar_obj <- subset( cel_seq_obj, cells=vano_acinar_cells)
gse85241_acinar_obj <- AddMetaData( gse85241_acinar_obj, 
                                  vano_aucell_dt[,.(cell.name,cell_type=paste0("GSE85241_",cell_type),
                                                   orig.ident="GSE85241")] %>%
                                  tibble::column_to_rownames("cell.name") ) %>% SetIdent(value="cell_type")

original_acinar_labelled_obj <- AddMetaData( original_acinar_obj, 
                                  edge_info_all$edge_center_dt[,.(cell.name,cell_type=fifelse(cell_category=="center","Reference_non_edge","Reference_edge"))] %>%
                                  mutate(orig.ident="Reference") %>% tibble::column_to_rownames("cell.name"))

obj_list <- list( gse81547_acinar_obj, gse85241_acinar_obj, original_acinar_labelled_obj )
for (i in 1:length(obj_list)) {
    obj_list[[i]] <- NormalizeData(obj_list[[i]], verbose = FALSE)
    obj_list[[i]] <- FindVariableFeatures(obj_list[[i]], selection.method = "vst", 
        nfeatures = 1000, verbose = FALSE)
}

anchors <- FindIntegrationAnchors(object.list = obj_list, dims = 1:30, anchor.features=1000)
integrated_acinar_obj <- IntegrateData(anchorset = anchors, dims = 1:30)
integrated_acinar_obj <- ScaleData( integrated_acinar_obj ) %>% RunPCA( .,npcs=30, verbose=F ) %>% 
RunUMAP(.,verbose=F,dims=1:30)

integrated_acinar_obj <- AddMetaData( integrated_acinar_obj,
                                    tibble::rownames_to_column(integrated_acinar_obj@meta.data,"cell.name") %>%
                                    mutate(merged_cell_type=ifelse(grepl("non_edge",cell_type),"merged_non_edge","merged_edge")) %>%
                                    tibble::column_to_rownames("cell.name"))

p_edge_merged_umap <- DimPlot( integrated_acinar_obj, group.by="orig.ident", 
        split.by="merged_cell_type",pt.size=1.5 ) + theme_pubr(base_size=13.33) + theme(legend.position="right")
rm(obj_list)

p_edge_merged_umap <- DimPlot( integrated_acinar_obj, group.by="orig.ident", 
        split.by="merged_cell_type",pt.size=1.5 ) + theme_pubr(base_size=13.33) + theme(legend.position="right")


### Figure 4G

In [ ]:
gse85241_acinar_obj <- NormalizeData( gse85241_acinar_obj ) %>% FindVariableFeatures(nfeatures=1000) %>% ScaleData(verbose=F) %>%
RunPCA(.,npcs=50,verbose=F)
pca_dist_info <- self_pca_distances( Cells(gse85241_acinar_obj), Embeddings(gse85241_acinar_obj,"pca") )
vano_pca_dist_df <- tibble::enframe( pca_dist_info$dist_from_medoid, name="cell.name", value="distance" ) %>%
merge(.,vano_aucell_dt[,.(cell.name,cell_type,display_cell_type=paste0("GSE85241\n",cell_type))])

gse81547_acinar_obj <- NormalizeData( gse81547_acinar_obj ) %>% FindVariableFeatures(nfeatures=1000) %>% ScaleData(verbose=F) %>%
RunPCA(.,npcs=50,verbose=F)
pca_dist_info <- self_pca_distances( Cells(gse81547_acinar_obj), Embeddings(gse81547_acinar_obj,"pca") )
efsun_pca_dist_df <- tibble::enframe( pca_dist_info$dist_from_medoid, name="cell.name", value="distance" ) %>%
merge(.,efsun_aucell_dt[,.(cell.name,cell_type,display_cell_type=paste0("GSE81547\n",cell_type))])

pca_dist_df <- rbind(vano_pca_dist_df,efsun_pca_dist_dfdist_df)

p_merged_edge_pca <- ggplot(pca_dist_df, aes(x=display_cell_type,y=distance) ) + geom_boxplot(aes(color=cell_type)) +
stat_compare_means(comparisons=list(c("GSE81547\nedge","GSE81547\nnon_edge"),
                                   c("GSE85241\nedge","GSE85241\nnon_edge"))) + theme_pubr(base_size=13.33) +
theme(legend.position="none")  + xlab(NULL) + ylab("Distance from medoid")

### Figure 4B

In [ ]:
combined_info_tbl <- rbind( efsun_info_tbl %>% mutate(data="GSE81547"), vano_info_tbl %>% mutate(data="GSE85241") )
edge_age_plot <- ggplot( combined_info_tbl,aes(x=age,y=frac_edge,color=data) ) + geom_point() + 
theme_pubr(base_size=13.33) +
theme(panel.grid.major=element_line(color="gray",linetype="dotted")) + 
xlab("Age") + ylab("Fraction of edge cells") + theme(legend.direction="vertical",
                                legend.position=c(0,1),legend.justification=c(0,1)) + labs(color=NULL)
cor_info <- cor.test(combined_info_tbl$frac_edge,combined_info_tbl$age)
cor_text <- paste0("R = ",round(cor_info$estimate,2),"\np = ",round(cor_info$p.value,2))
edge_age_plot <- edge_age_plot + annotation_custom( grobTree(textGrob(cor_text, x=0.4,  y=0.9, hjust=0,
   gp=gpar(col="black", fontsize=10, fontface="italic"))) )

## Checking distribution of mutations between edge and non-edge cells

In [ ]:
gse81547_mutations_per_cell_df <- load_GSE81547_mutations(efsun_aucell_dt)
detach("package:monocle3")
gse85241_mutations_per_cell_df <- process_GSE85241_mutations(vano_aucell_dt)
library(monocle3)

In [ ]:
#GSE85241
gse85241_box_plot <- ggplot( gse85241_mutations_per_cell_df %>% dplyr::group_by(cell.name,.keep=cell_type) %>% dplyr::count(name="n_total")) + 
geom_boxplot(aes(x=.keep,y=n_total)) + theme_pubr(base_size=10) +
stat_compare_means(aes(x=.keep,y=n_total),label="..p.signif..",size=4,label.x=1.5) + ylab("# mutations") + 
scale_x_discrete(labels=c("non_edge"="Non-Edge","edge"="Edge")) + xlab(NULL)

p_values <- c()
for (sub_samp in 1:100) {
    cells_to_retain <- subsample_edge_non_edge_cells( vano_aucell_dt )
    set.seed(sub_samp)
    cell_type_wise_num_mutations_dt <- vano_aucell_dt[cell.name %in% cells_to_retain,.N,by=list(age,cell_type)] %>% setnames(.,"N","n_total")

    mut_freq_dt <- gse85241_mutations_per_cell_df %>% 
    dplyr::group_by(age,mutation,cell_type,.drop=F) %>% 
    dplyr::count(name="n_mut") %>% merge(.,cell_type_wise_num_mutations_dt,by=c("age","cell_type")) %>%
    mutate(mut_freq=n_mut/n_total) %>% data.table
    
    p_values <- c(p_values,wilcox.test(mut_freq_dt[cell_type == "non_edge",mut_freq],
        mut_freq_dt[cell_type == "edge",mut_freq],alternative="l" )$p.value)
}

cell_type_wise_num_mutations_dt <- vano_aucell_dt[,.N,by=list(age,cell_type)] %>% setnames(.,"N","n_total")
mut_freq_dt <- gse85241_mutations_per_cell_df %>% 
dplyr::group_by(age,mutation,cell_type,.drop=F) %>% 
dplyr::count(name="n_mut") %>% merge(.,cell_type_wise_num_mutations_dt,by=c("age","cell_type")) %>%
mutate(mut_freq=n_mut/n_total,cell_type=paste("GSE85241",cell_type)) %>%
arrange(-mut_freq) %>% data.table# merge(edge_info_dt[cell_type =="edge",!c("cell_type")],by="age") %>% 

options(repr.plot.height=4,repr.plot.width=6)
gse85241_hist_plot <- ggplot( mut_freq_dt, aes(x=mut_freq)) + 
geom_histogram(aes(y=..count../sum(..count..)),bins=20) + facet_wrap(~cell_type) +
ylab("Frequency") + theme_pubr(base_size=13.33) + xlab("Fraction of cells with somatic mutation")

In [ ]:
mut_freq_dt %>% group_by(cell_type) %>% summarize(m=mean(mut_freq))

In [ ]:
#GSE81547
combined_df <- gse85241_mutations_per_cell_df %>% 
dplyr::group_by(cell.name,.keep=cell_type) %>% dplyr::count(name="n_total") %>%
mutate(cell_type=paste("GSE85241\n",.keep)) %>%
rbind(.,gse81547_mutations_per_cell_df %>% dplyr::rename(cell.name=cellIDxmutID) %>%
       dplyr::group_by(cell.name,.keep=cell_type) %>% 
       dplyr::count(name="n_total") %>% mutate(cell_type=paste("GSE81547\n",.keep)))

combined_box_plot <- ggplot(combined_df,aes(x=cell_type,y=n_total)) + 
geom_boxplot(aes(color=.keep)) + theme_pubr(base_size=13.33) +
stat_compare_means(comparisons=list(c("GSE81547\n edge","GSE81547\n non_edge"),
                                    c("GSE85241\n edge","GSE85241\n non_edge")),
                                    label="p.signif",size=4,label.x=1.5) + ylab("# mutations") + 
# scale_x_discrete(labels=c("GSE81547\n non_edge"="GSE81547","GSE81547\n edge"="GSE81547",
#                          "GSE85241\n non_edge"="GSE85241","GSE85241\n edge"="GSE85241")) + 
xlab(NULL) + theme(legend.position="none")
cell_type_wise_num_mutations_dt <- efsun_aucell_dt[,.N,by=list(age,cell_type)] %>% setnames(.,"N","n_total")

p_values <- c()
for (sub_samp in 1:100) {
    cells_to_retain <- subsample_edge_non_edge_cells( efsun_aucell_dt )
    set.seed(sub_samp)
    cell_type_wise_num_mutations_dt <- efsun_aucell_dt[cell.name %in% cells_to_retain,.N,by=list(age,cell_type)] %>% setnames(.,"N","n_total")

     mut_freq_dt <- gse81547_mutations_per_cell_df %>% dplyr::group_by(age,name,cell_type,.drop=F) %>% 
    dplyr::count(name="n_mut") %>% merge(.,cell_type_wise_num_mutations_dt,
                                         by=c("age","cell_type")) %>%
    mutate(mut_freq=n_mut/n_total) %>% arrange(-mut_freq) %>% data.table

    p_values <- c(p_values,wilcox.test(mut_freq_dt[cell_type == "non_edge",mut_freq],
        mut_freq_dt[cell_type == "edge",mut_freq],alternative="l" )$p.value)
}


mut_freq_dt <- gse81547_mutations_per_cell_df %>% dplyr::group_by(age,name,cell_type,.drop=F) %>% 
dplyr::count(name="n_mut") %>% merge(.,cell_type_wise_num_mutations_dt,
                                     by=c("age","cell_type")) %>% dplyr::filter(n_total >= 10) %>%
mutate(mut_freq=n_mut/n_total,cell_type=paste("GSE81547",cell_type)) %>% arrange(-mut_freq) %>% data.table 

gse81547_hist_plot <- ggplot( mut_freq_dt, aes(x=mut_freq)) + 
geom_histogram(aes(y=..count../sum(..count..)),bins=20) + facet_wrap(~cell_type) +
ylab("Frequency") + theme_pubr(base_size=13.33) + xlab("Fraction of cells with somatic mutation")

In [ ]:
mut_freq_dt %>% group_by(cell_type) %>% summarize(m=mean(mut_freq))

### Check classification of mutations in COSMIC Cancer Gene Census

In [ ]:
results_list <- list()
accession_ids <- c("GSE81547","GSE85241")
cosmic_dt <- fread(file.path(base_path,"CosmicCodingMuts.hg19_ucsc.v92.vcf.gz"))
cosmic_census_dt <- fread(file.path(base_path,"cmc_export.v92.tsv.gz"))
source("../src/process_pdac.R")
idx <- 1
for (df in list(gse81547_mutations_per_cell_df,gse85241_mutations_per_cell_df)) {
    print(accession_ids[idx])
    if ("name" %in% colnames(df)) {
        mutations <- df$name %>% unique
    } else {
        mutations <- df$mutation %>% unique
    }
    results_list[[accession_ids[idx]]] <- search_cosmic_database( cosmic_dt, cosmic_census_dt, mutations )
    idx <- idx + 1
}

### Create Figure 4

In [ ]:
options(repr.plot.width=10,repr.plot.height=6.5)
Fig4_complete <- grid.arrange( arrangeGrob(grobs=
                          list(p_efsun_markers + theme(axis.text.x=element_text(angle=90,hjust=1,vjust=0.5)
                                                                    ) + ggtitle("GSE81457"), 
                                     p_vano_markers + theme(axis.text.x=element_text(angle=90,hjust=1,vjust=0.5),
                                                           axis.text.y=element_blank()) + ggtitle("GSE85241"),
          edge_age_plot,combined_box_plot,
                                     gse81547_hist_plot + theme(axis.text.x=element_text(angle=90,hjust=1,vjust=0.5)),
          gse85241_hist_plot + theme(axis.text.x=element_text(angle=90,hjust=1,vjust=0.5)),
                                           p_edge_merged_umap + theme(legend.position="none"),
                                           p_merged_edge_pca),
                         nrow=3,ncol=4,
                           widths=c(0.75,0.5,1,1),
                           layout_matrix = rbind(c(1, 2, 3, 4),
                                                 c(1, 2, 5, 6),
                                          c(7,7,7,8)) ))
ggsave("Fig4_complete_new.svg",Fig4_complete,width=12,height=7.5)

## Lung data

In [ ]:
source("../src/process_pdac.R")
lung_data <- load_lung_data()
for (normal_cell_type in c("Ciliated","Club","AT1","AT2")) {
    lung_edge_info <- run_edge_pipeline(lung_data$seurat_obj,normal_cell_types=normal_cell_type,
                  malignant_cell_types="tS2",pipeline_variant = "feature-selection",ident_to_use="cluster",
                                   sample_info_column="Sample", num_pcs=5)
}

lung_edge_info_list <- run_pc_wise_edge_analysis( lung_data$seurat_obj, normal_cell_types=c("Ciliated","Club","AT1","AT2"),
                                                 malignant_cell_type="tS2",ident_to_use="cluster",
                                                 sample_info_column="Sample")
mascaux_gene_sets <- c("MYC_TARGETS_V2","MYC_TARGETS_V1","MTORC1_SIGNALING","PI3K_AKT_MTOR_SIGNALING",
                      "E2F_TARGETS","MITOTIC_SPINDLE","INTERFERON_GAMMA_RESPONSE","ALLOGRAFT_REJECTION",
                      "G2M_CHECKPOINT","IL6_JAK_STAT3_SIGNALING","TNFA_SIGNALING_VIA_NFKB","INFLAMMATORY_RESPONSE",
                      "IL2_STAT5_SIGNALING","ADIPOGENESIS","EPITHELIAL_MESENCHYMAL_TRANSITION","UV_RESPONSE_UP","UV_RESPONSE_DN",
                      "OXIDATIVE_PHOSPHORYLATION","FATTY_ACID_METABOLISM","HYPOXIA")


## Liver data

In [ ]:
liver_data <- load_liver_data()
for (normal_cell_type in paste0("Hep",1:6)) {
    liver_edge_info <- run_edge_pipeline(liver_data$seurat_obj,normal_cell_types=normal_cell_type,
                   malignant_cell_types="Malignant cell",pipeline_variant = "feature-selection",
                                      num_pcs=5, ident_to_use="Type",edge_cell_fraction=0.1,
                                    sample_info_column="Sample")
}
liver_edge_info_list <- run_pc_wise_edge_analysis( liver_data$seurat_obj, normal_cell_types=paste0("Hep",1:6),
                                                 malignant_cell_type="Malignant cell",ident_to_use="Type",
                                                 sample_info_column="Sample")

p-values of enrichment of Mascaux et. al gene sets amongst each of the edge-like populations

## Breast

### Mouse

In [ ]:
mouse_obj <- readRDS(file.path(base_path,"BRCA1_SCE.rds"))

In [ ]:
tumor_df <- temp_df %>% mutate(idx=1:nrow(temp_df)) %>% dplyr::filter(Groups %in% c("Luminal","TumorEpithel") &
                                                                   Experiment == "Tumorigenesis")
count_mat <- counts(mouse_obj)[,tumor_df$idx]
colnames(count_mat) <- tumor_df$barcode

In [ ]:
mouse_seurat_obj <- CreateSeuratObject( count_mat,
                                       meta.data=temp_df %>% tibble::column_to_rownames("barcode") %>% 
                                       dplyr::filter(Groups %in% c("Luminal","TumorEpithel") &
                                                                   Experiment == "Tumorigenesis") )

In [ ]:
tumor_df$Groups %>% unique

In [ ]:
source("../src/process_pdac.R")
mouse_seurat_obj <- NormalizeData(mouse_seurat_obj)
mouse_luminal_info <- run_edge_pipeline( mouse_seurat_obj, malignant_cell_types = c("Tm"),
                              normal_cell_types=c("Hsp","Hs","LpBsl","Avd","Lp"), num_pcs=5, num_control_shuffles=100,
                                                            pipeline_variant="classic",
                              perform_control=T, no_tumour_adjacent=F, sample_info_column="SampleID",
                                               ident_to_use="CellTypesFinal")


In [ ]:
options(repr.plot.width=12)
plot_skewness(mouse_luminal_info$control_dist_dt,plot_facets = T)$plot_obj
plot_edge_distance_ratio(mouse_luminal_info$edge_malignant_dist_dt,plot_facets = T)$plot_obj
options(repr.plot.width=7)

In [ ]:
plot_df <- merge(mouse_luminal_info$edge_center_dt, tumor_df,by.x="cell.name",by.y="barcode")

In [ ]:
colnames(plot_df)

In [ ]:
ggplot( plot_df %>% dplyr::filter(CellTypesFinal == "Hsp")) + 
geom_boxplot(aes(x=Condition,y=dist_from_malignant_medoid)) + theme_pubr(base_size=15)

In [ ]:
ggplot( mouse_luminal_info$edge_center_dt ) + geom_point(aes(x=dist_from_normal_medoid,
                                                            y=dist_from_malignant_medoid,
                                                            color=cell_category)) + theme_pubr(base_size=15)

In [ ]:
ggplot( temp_df ) + geom_point(aes(x=UMAP1,y=UMAP2,color=Groups))

### Human

In [ ]:
source("../src/process_pdac.R")
bc_obj <- load_breast_cancer_data()# %>% subset(.,subset = CellType == "Cancer") #%>% subset(.,subset=nFeature_RNA >= 1500)
# normal_breast_obj <- load_normal_breast_data()

bc_tumor_counts_df <- c("BC_1"=3921,"BC_2"=4876,"BC_3"=4251,"BC_4"=3558,
                   "BC_5"=4472,"BC_6"=201,"BC_7"=1719,"BC_8"=2427,
                   "BC_9"=4616,"BC_10"=902,"BC_11"=4862,"BC_12_1"=362,
                   "BC_12_2"=64,"BC_12_3"=1372,"BC_13"=3641,"BC_14"=4216,
                   "BC_15_1"=4143,"BC_15_2"=347,"BC_16"=269) %>% 
tibble::enframe(.,name="Tumor_Label",value="n")

bc_tumor_types_df <- c("BC_1"="HER2+","BC_2"="TN","BC_3"="TN","BC_4"="TN",
                   "BC_5"="TN","BC_6"="HER2+","BC_7"="TN","BC_8"="HER2+",
                   "BC_9"="Luminal-HER2+","BC_10"="TN","BC_11"="TN","BC_12_1"="TN",
                   "BC_12_2"="TN","BC_12_3"="TN","BC_13"="Luminal-B-like","BC_14"="Luminal-A-like",
                   "BC_15_1"="TN","BC_15_2"="TN","BC_16"="Luminal-B-like") %>%
tibble::enframe(.,name="Tumor_Label",value="Cancer_Type")
cancer_types <- unique(bc_tumor_types_df$Cancer_Type)

seurat_tumor_counts_df <- bc_obj@meta.data %>% group_by(PatientNumber) %>% dplyr::count() %>%
merge(.,bc_tumor_counts_df,by="n") %>% merge(.,bc_tumor_types_df,by="Tumor_Label") %>% dplyr::select(-n)

meta_data_df <- merge( tibble::rownames_to_column(bc_obj@meta.data,"cell.name"), 
      seurat_tumor_counts_df, by="PatientNumber" ) %>%
mutate(CellType=ifelse(CellType == "Cancer",Cancer_Type,CellType)) %>%
tibble::column_to_rownames("cell.name")
bc_obj <- AddMetaData( bc_obj, meta_data_df )

bc_obj <- subset(bc_obj,subset=CellType %in% unique(bc_tumor_types_df$Cancer_Type) &
                nFeature_RNA >= 1000)

In [ ]:
source("../src/process_pdac.R")
normal_breast_fl_obj <- load_normal_breast_fluidigm_data() %>% subset(.,subset=nFeature_RNA >= 900)
normal_breast_10x_obj <- load_normal_breast_10X_data() %>% subset(cluster %in% c("Basal","Luminal1.1","Luminal1.2",
                                                             "Luminal2","Myoepithelial")) %>% subset(.,subset=nFeature_RNA >= 1000)

normal_breast_10x_obj$CellType <- normal_breast_10x_obj$cluster
normal_breast_10x_obj$PatientNumber <- normal_breast_10x_obj$orig.ident
merged_obj <- merge(bc_obj,normal_breast_10x_obj)

In [ ]:
source("../src/process_pdac.R")
feature_counts_to_subsample <- merged_obj@meta.data %>% dplyr::filter(!CellType %in% cancer_types) %>% 
dplyr::select(nCount_RNA) %>% tibble::rownames_to_column("cell.name") %>% tibble::deframe(.)
feature_counts_reference <- merged_obj@meta.data %>% dplyr::filter(CellType %in% cancer_types) %>% 
dplyr::select(nCount_RNA) %>% tibble::rownames_to_column("cell.name") %>% tibble::deframe(.)
set.seed(1)
subsampled_obj <- create_fast_resampled_seurat_obj(merged_obj[["RNA"]]@counts,
                                                  feature_counts_to_subsample,
                                                  feature_counts_reference,
                                                  feature_to_subsample="reads" )
merge(tibble::rownames_to_column(subsampled_obj@meta.data,"cell.name"),
tibble::rownames_to_column(merged_obj@meta.data,"cell.name") %>% 
      dplyr::select(-c(nCount_RNA,nFeature_RNA,orig.ident)),
by="cell.name") %>% tibble::column_to_rownames("cell.name") -> meta_data_df
subsampled_obj <- AddMetaData(subsampled_obj,meta_data_df)

In [ ]:
source("../src/process_pdac.R")
malignant_cell_types <- cancer_types
normal_cell_types <- setdiff( unique(normal_breast_10x_obj$CellType), malignant_cell_types)
breast_edge_info_list <- list()
for (malignant_cell_type in malignant_cell_types) {
    for (cell_type in normal_cell_types){
        breast_edge_info_list[[paste(cell_type,malignant_cell_type,sep="_")]] <- run_edge_pipeline( subsampled_obj %>%
                                                            subset(.,subset = CellType %in% c(cell_type,malignant_cell_types)), malignant_cell_types=malignant_cell_types, 
                              normal_cell_types=c(cell_type), num_pcs=5, num_control_shuffles=100,
                                                            pipeline_variant="feature-selection",
                              perform_control=T, no_tumour_adjacent=F, sample_info_column="PatientNumber",
                                               ident_to_use="CellType")
#     4 + ""
    }
}


In [ ]:
breast_edge_info_list[['Myoepithelial_TN']]$pca_cor_df %>% head

In [ ]:
names(breast_edge_info_list)

In [ ]:
temp <- readRDS("sample_aware_Luminal2_Luminal-B-like_RNA_analysis.rds")

In [ ]:
temp

In [ ]:
normal_breast_10x_obj <- SetIdent(normal_breast_10x_obj,value="cluster")

## Colon

### Load normal colon data

In [ ]:
source("../src/process_pdac.R")
colon_obj <- load_colon_data()#readRDS(file.path(base_path,"Colon_Seurat_Object.rds"))
colon_obj <- subset(colon_obj,subset=nFeature_RNA >= 1000)

### Load colon cancer data

In [ ]:
crc_obj <- load_generic_cancer_data(cancer_type = "CRC") %>% subset(.,subset=nFeature_RNA>=1000 & CellType == "Cancer" )
colon_obj$CellType <- colon_obj$Cluster
merged_obj <- merge(colon_obj,crc_obj)
genes_to_keep <- names(which(rowMeans(merged_obj[["RNA"]]@counts) >= 0.01))
merged_obj <- subset(merged_obj,features=genes_to_keep)

### Sub-sample normal colon data to match library size of colon cancer data

In [ ]:
source("../src/process_pdac.R")
feature_counts_to_subsample <- merged_obj@meta.data %>% dplyr::filter(CellType != "Cancer") %>% 
dplyr::select(nCount_RNA) %>% tibble::rownames_to_column("cell.name") %>% tibble::deframe(.)
feature_counts_reference <- merged_obj@meta.data %>% dplyr::filter(CellType == "Cancer") %>% 
dplyr::select(nCount_RNA) %>% tibble::rownames_to_column("cell.name") %>% tibble::deframe(.)
set.seed(1)
subsampled_obj <- create_fast_resampled_seurat_obj(merged_obj[["RNA"]]@counts,
                                                  feature_counts_to_subsample,
                                                  feature_counts_reference,
                                                  feature_to_subsample="reads" )
merge(tibble::rownames_to_column(subsampled_obj@meta.data,"cell.name"),
tibble::rownames_to_column(merged_obj@meta.data,"cell.name") %>% 
      dplyr::select(-c(nCount_RNA,nFeature_RNA,orig.ident)),
by="cell.name") %>% tibble::column_to_rownames("cell.name") -> meta_data_df
subsampled_obj <- AddMetaData(subsampled_obj,meta_data_df)

In [ ]:
source("../src/process_pdac.R")
malignant_cell_types <- c("Cancer")
normal_cell_types <- setdiff( unique(subsampled_obj$CellType), malignant_cell_types)
colon_edge_info_list <- list()
for (cell_type in normal_cell_types){
    colon_edge_info_list[[cell_type]] <- run_edge_pipeline( subsampled_obj,
                                                           malignant_cell_type="Cancer",
                                              normal_cell_types=cell_type, 
                                                           num_pcs=5, num_control_shuffles=100,
                                                            pipeline_variant="feature-selection",
                              perform_control=T, no_tumour_adjacent=F, sample_info_column="Subject",
                                               ident_to_use="CellType")
}
edge_cell_types <- names(colon_edge_info_list)
colon_edge_info_list <-  run_pc_wise_edge_analysis( subsampled_obj, 
                                                   normal_cell_types=edge_cell_types,
                                                 malignant_cell_type="Cancer",ident_to_use="CellType",
                                                 sample_info_column="Subject")

## Figure 5A display

In [ ]:
lung_edge_features <- readRDS("sample_aware_AT2_tS2_RNA_analysis.rds")

skewness_dt <- lung_edge_features$skewness_p_value[order(-p_value),] %>% 
mutate(Var1=gsub("normal_","",Var1) %>% gsub("_","",.)) %>% tail(10)     
skewness_dt$Var1 <- factor(skewness_dt$Var1,levels=skewness_dt$Var1)
p_skew <- ggplot( skewness_dt[,significant:=fifelse(q_value < 0.1,TRUE,FALSE)] ) + 
geom_bar(aes(x=Var1,y=value,fill=significant),stat="identity") + coord_flip() +
theme_pubr(base_size=10) + xlab("Normal PC") + ylab("Skewness")

edge_distance_dt <- lung_edge_features$edge_center_p_value[normal_pc == "normal_PC_1",][order(-p_value),] %>% 
mutate(pc=gsub("combined_","",pc) %>% gsub("_","",.)) %>% tail(10)     
edge_distance_dt$pc <- factor(edge_distance_dt$pc,levels=edge_distance_dt$pc)
p_ratio <- ggplot( edge_distance_dt[,significant:=fifelse(q_value < 0.1,TRUE,FALSE)] ) + 
geom_bar(aes(x=pc,y=edge_center_dist_ratio,fill=significant),stat="identity") + coord_flip() +
theme_pubr(base_size=10) + xlab("Pooled PC") + ylab("Proximity Ratio\n(Outliers defined using Normal PC5)")

main_pca_cor_df <- get_pca_correlations( lung_edge_info$normal_pca, lung_edge_info$combined_pca ) %>% 
dplyr::select(Var1,Var2,cor,q_value) %>% dplyr::filter(Var1=="normal_PC_5") %>%
mutate(Var1=gsub("normal_","",Var1) %>% gsub("_","",.),
       Var2=gsub("combined_","",Var2) %>% gsub("_","",.) ) %>%
mutate(significant=ifelse(q_value < 0.1,TRUE,FALSE)) %>% dplyr::filter(Var2 %in% paste0("PC",1:10))

p_pca_cor <- ggplot( main_pca_cor_df %>% mutate(significant=ifelse(q_value < 0.1,TRUE,FALSE) )) + 
geom_bar(aes(x=Var2,y=cor,fill=significant),stat="identity") + coord_flip() +
theme_pubr(base_size=10) + xlab("Normal PC") + ylab("Corr. with Normal PC5")

fig5a <- ggarrange( p_skew + theme(legend.position="none"), 
          p_ratio + theme(legend.position="none"), 
          p_pca_cor + theme(legend.position="none"), nrow=3, ncol=1 )
ggsave("Fig5a.svg",fig5a,width=3,height=6.5)

## Tosti et. al 2020 pancreas data

### Spatial transcriptomic data

In [ ]:
spatial_files <- c("AFES448-FF2_reads.csv",
                   "AFHES365-H1_reads.csv",
                   "AGBR024-B2_reads.csv",
                   "AGBRO24-H1_reads.csv",
                   "TUM-CP-2_reads.csv")
lr_markers_df_list <- readRDS("all_outlier_markers.rds")
edge_acinar_markers_df <- lr_markers_df_list[["Acinar cell"]] %>% dplyr::filter(p_val_adj < 0.1 & avg_logFC > 0)
plot_list <- list()
p_value_list <- list()

num_subsamples <- 100
sample_size <- 5000
num_edge_assignments <- 100

for (spatial_file in spatial_files) {
    reads_dt <- fread(file.path(base_path,spatial_file))

    spatial_genes <- unique(reads_dt$gene)
    spatial_edge_genes <- c(spatial_genes[spatial_genes %in% edge_acinar_markers_df$gene],"MUC5B")


    distances_df <- data.frame()
    idx <- 1
    for (random.seed in 1:num_subsamples) {
        print(idx)
        flush.console()
        set.seed(idx)
        for (gene_ in spatial_genes) {
            coords_mat <- reads_dt %>% dplyr::filter(gene == gene_) %>% dplyr::select(-gene) %>% as.matrix
            if (nrow(coords_mat) > sample_size) {
                idxes <- sample(nrow(coords_mat),size=sample_size)
            } else {
                idxes <- 1:nrow(coords_mat)
            }
            distances <- as.vector( parDist(coords_mat[idxes,], threads=6 ))
            distances_df <- rbind(distances_df,data.frame(gene=gene_,distance=median(distances),
                                                         sample_idx=idx))
        }
        idx <- idx + 1
    }
    distances_df <- distances_df %>% mutate(gene_type=ifelse(gene %in% spatial_edge_genes,"Edge","Non-Edge"))

    median_diff_df <- data.frame()
    for (edge_assignment in 1:num_edge_assignments) {
        random_distances <- distances_df %>% 
        mutate(gene_type=ifelse(gene %in% sample(spatial_genes,10),"Edge","Non-Edge")) %>%
        group_by(gene_type) %>% summarize(m=median(distance)) %>% ungroup %>% tibble::deframe(.)
        median_diff_df <- rbind(median_diff_df,data.frame(diff=random_distances["Edge"] - random_distances["Non-Edge"],
                                                         shuffle_type="Control"))
    }
    actual_distances <- distances_df %>% group_by(gene_type) %>% summarize(m=median(distance)) %>% ungroup %>% tibble::deframe(.)
    actual_diff <- actual_distances["Edge"] - actual_distances["Non-Edge"]

    m <- mean(median_diff_df$diff)
    s <- sd(median_diff_df$diff)
    p_value <- pnorm(actual_diff,m=m,s=s)
    p_value_list[[spatial_file]] <- p_value

    plot_list[[spatial_file]] <- ggplot( median_diff_df ) + geom_density(aes(x=diff),color="gray") + 
    geom_vline(data=data.frame(m=m), xintercept=m, color="black") + theme_pubr(base_size=13.33)
}

### Figure 6D

In [ ]:
options(repr.plot.width=10)
plot_idx <- 1
for (plot_name in names(plot_list)) {
    print(plot_name)
    plot_list[[plot_name]] <- plot_list[[plot_name]] + ylab(NULL) + ylim(0,0.003) + xlab(NULL) + theme_pubr(base_size=13.33)
    if (plot_idx > 1) {
        plot_list[[plot_name]] <- plot_list[[plot_name]] + theme(axis.text.y=element_blank())
    }
    plot_idx <- plot_idx + 1
}
spatial_plots <- ggarrange(plotlist=plot_list,ncol=5)
# options(repr.plot.width=7)
ggsave("Fig6D.svg",spatial_plots,width=12,height=2.8)

### Single-nucleus data

In [ ]:
source("../src/process_pdac.R")
tosti_combined_seurat_obj <- load_tosti2020_combined_data() 

lr_markers_df_list <- readRDS("all_outlier_markers.rds")
edge_acinar_markers_df <- lr_markers_df_list[["Acinar cell"]] %>% dplyr::filter(p_val_adj < 0.1)

edge_gene_set <- list("edge"=edge_acinar_markers_df %>% dplyr::filter(avg_logFC > 0) %>% pull(gene))    

epithelial_obj <- subset(tosti_combined_seurat_obj,subset = Cluster %in% 
                           c("Acinar-s","Acinar-i","Acinar-REG+","MUC5B+ Ductal"))
rm(tosti_combined_seurat_obj)

genes_to_keep <- names(which(rowMeans(epithelial_obj[["RNA"]]@counts > 0) > 0.01))

### Figure 6A

In [ ]:
markers <- c("AMBP","CFTR","MMP7","ANXA4", "SLC4A4", "PRSS1", "CTRB1", "CTRB2", "REG1B")
# tosti_combined_seurat_obj <- NormalizeData(tosti_combined_seurat_obj)
p_dot_plot <- DotPlot( subset( epithelial_obj, 
                subset = Cluster %in% c("Acinar-s","Acinar-i","Acinar-REG+","MUC5B+ Ductal","Ductal")),
        features=markers ) + theme(axis.text.x=element_text(angle=90,hjust=1,vjust=0.5))
muc_ductal_cells <- WhichCells(epithelial_obj,expression = Cluster == "MUC5B+ Ductal")
for (marker in c("PRSS1","CTRB1","CTRB2","REG1B")) {
    pct_expressed <- mean( epithelial_obj[["RNA"]]@counts[marker,muc_ductal_cells] > 0)
    print(paste(marker,pct_expressed))
}

### Figure 6B, 6C

In [ ]:
samples_to_use <- c('AFES448-head','AFES448-body','AFES448-tail','AFES365-body','AFES365-head','AFES365-tail','AGBR024-body',
'AGBR024-head','AGBR024-tail','TUM_13-tail','TUM_25-body','TUM_C1-body','TUM_CP1-head','TUM_CP2-head')
ret_auc <- compute_AUCell_scores(subset( epithelial_obj, subset = orig.ident %in% samples_to_use,
                                       features=genes_to_keep )
,edge_gene_set,compute_thresholds=F,nCores=3)
merge( tibble::rownames_to_column(as.data.frame(ret_auc$auc_mat),"cell.name"),
      tibble::rownames_to_column(epithelial_obj@meta.data,"cell.name") %>% dplyr::select(cell.name,Cluster,sample=orig.ident), 
      by="cell.name" ) -> merged_df

control_sd_df <- compute_shuffled_gene_set_AUCell_scores(subset( epithelial_obj, 
                                                                subset = orig.ident %in% samples_to_use,
                                       features=genes_to_keep ), edge_gene_set, q_thresh=0.95)
threshold_df <- control_sd_df %>% group_by(sample) %>% summarize(thresh=max(mean))
merged_df <- merge( merged_df, threshold_df, by="sample" ) %>% mutate(is_edge=ifelse(edge > thresh,1,0)) %>%
mutate(sample_type=ifelse(grepl("^A",sample) | !grepl("CP",sample),"Healthy","CP"))
fwrite(merged_df,"Tosti_Edge_AUC.tsv",sep="\t")

In [ ]:
merged_df <- fread("Tosti_Edge_AUC.tsv")
cell_types <- unique(epithelial_obj$Cluster)
merged_pca_dist_df <- data.frame()
for (cell_type in cell_types) {
    for (sample_type_ in c("Healthy","CP") ) {
        cells <- merged_df %>% dplyr::filter(sample_type == sample_type_ & Cluster == cell_type) %>% pull(cell.name)
        flush.console()
        if (length(cells) < 20)
            next

        print(cell_type)
        print(sample_type_)

        subset_obj <- subset(epithelial_obj, cells=cells) %>% FindVariableFeatures %>%
        ScaleData(.,verbose=F) %>% RunPCA(.,npcs=20,verbose=F)
        pca_dist_info <- self_pca_distances( Cells(subset_obj), Embeddings(subset_obj,"pca") )
        pca_dist_df <- tibble::enframe( pca_dist_info$dist_from_medoid, name="cell.name", value="distance" ) %>%
    merge( ., merged_df ) %>% mutate(is_edge=factor(is_edge,levels=c(1,0)),
                                     sample_type=factor(sample_type,levels=c("Healthy","CP")))
        merged_pca_dist_df <- rbind(merged_pca_dist_df,pca_dist_df)
        wilcox.test(pca_dist_df %>% dplyr::filter(is_edge == 1) %>% pull(distance),
                   pca_dist_df %>% dplyr::filter(is_edge == 0) %>% pull(distance),alternative="g")
    }
}
fwrite(merged_pca_dist_df,"Tosti_Edge_AUC_PCA_Distances.tsv")

In [ ]:
merged_pca_dist_df <- fread("Tosti_Edge_AUC_PCA_Distances.tsv")#,sep="\t")
merged_df <- fread("Tosti_Edge_AUC.tsv")

for (sample_type_ in c("Healthy","CP")) {
    for (cell_type in unique(merged_df$Cluster)) {
        pca_dist_df <- merged_pca_dist_df %>% dplyr::filter(Cluster == cell_type & sample_type == sample_type_)
        if (nrow(pca_dist_df) == 0)
            next
        print(sample_type_)
        print(cell_type)

        print(wilcox.test(pca_dist_df %>% dplyr::filter(is_edge == 1) %>% pull(distance),
                   pca_dist_df %>% dplyr::filter(is_edge == 0) %>% pull(distance),alternative="g"))
        flush.console()
    }
}
merged_pca_dist_df$is_edge <- factor(merged_pca_dist_df$is_edge,levels=c(1,0))
merged_pca_dist_df$sample_type <- factor(merged_pca_dist_df$sample_type,levels=c("Healthy","CP"))
p_pca_dist_plot <- ggplot( merged_pca_dist_df ) + geom_boxplot(aes(x=Cluster,y=distance,color=is_edge),
                                           outlier.shape=NA) + facet_wrap(~sample_type) +
theme_pubr(base_size=13.33) + ylim(0,65)

p_cp_boxplot <- ggplot( merged_df ) + 
geom_boxplot(aes(x=Cluster,y=edge,color=sample_type),outlier.shape=NA,
            position=position_dodge(preserve="single")) + labs(color=NULL) + theme_pubr(base_size=12) + 
xlab("AUCell")

In [ ]:
cell_counts_dt <- merged_df %>% mutate(sample_type=ifelse(grepl("^A",sample) | !grepl("CP",sample),"Healthy",
                                     "CP")) %>% mutate(Cluster=sample_type) %>% 
group_by(Cluster,is_edge) %>% dplyr::count() %>% 
mutate(is_edge=ifelse(is_edge==1,"edge","non-edge")) %>% dplyr::rename(cell_type=is_edge,N=n) %>% data.table
for (cluster in unique(cell_counts_dt$Cluster)) {
    cell_type_dt <- cell_counts_dt[Cluster == cluster,]
    num_cells <- sum(cell_type_dt$N)
    num_is_type_is_edge <- cell_type_dt[cell_type == "edge",N]
    num_is_type_is_non_edge <- cell_type_dt[cell_type != "edge",N]
    num_is_not_type_is_edge <- sum(cell_counts_dt[cell_type == "edge" & Cluster != cluster,N])
    num_is_not_type_is_non_edge <- sum(cell_counts_dt[cell_type != "edge" & Cluster != cluster,N])
    fisher_mat <- matrix(c(num_is_type_is_edge,num_is_type_is_non_edge,
                          num_is_not_type_is_edge,num_is_not_type_is_non_edge),
                        nrow=2,ncol=2,byrow=T)

    fisher_res <- fisher.test(fisher_mat,alternative="g")
    print(cluster)
#     print(fisher_mat)
    print(fisher_res$p.value)
    print(fisher_res$estimate)
    flush.console()
}

### Figure 6 merging

In [ ]:
options(repr.plot.height=5,repr.plot.width=15)
fig6 <- grid.arrange(
    arrangeGrob(p_dot_plot + labs(tag="A") + theme(axis.text.x=element_text(angle=0)) + xlab(NULL),
    p_cp_boxplot + theme(legend.position="none") + xlab(NULL) + ylab("AUCell") + labs(tag="B") +
                scale_color_manual(values=c("Healthy"="black","CP"="gray")),
                                                       p_pca_dist_plot + labs(tag="C") + xlab(NULL) +
                ylab("Distance from medoid") + theme(legend.position="none"),
     widths=c(1.75,1,1),
    ncol=3,nrow=1
))
saveRDS(fig6,"Fig6.rds")
ggsave("Fig6_shortened.svg",fig6,width=12,height=2.5)

## Human prostate

In [ ]:
source("../src/process_pdac.R")
prostate_obj <- load_prostate_data()

In [ ]:
source("../src/process_pdac.R")
malignant_cell_types <- c("Epi_Luminal_1_Malignant")
normal_cell_types <- c("Epi_Basal_1","Epi_Basal_hillock","Epi_Luminal_1","Epi_Luminal_2")
prostate_edge_info_list <- list()
for (cell_type in normal_cell_types){
    prostate_edge_info_list[[cell_type]] <- run_edge_pipeline( prostate_obj,
                                                           malignant_cell_type="Epi_Luminal_1_Malignant",
                                              normal_cell_types=cell_type, 
                                                           num_pcs=20, num_control_shuffles=100,
                                                            pipeline_variant="feature-selection",
                              perform_control=T, no_tumour_adjacent=F, sample_info_column="orig.ident",
                                               ident_to_use="fullTypeTumorPred")

}
edge_cell_types <- names(prostate_edge_info_list)
prostate_edge_info_list <-  run_pc_wise_edge_analysis( prostate_obj, 
                                                   normal_cell_types=edge_cell_types,
                                                 malignant_cell_type="Epi_Luminal_1_Malignant",
                                                      ident_to_use="fullTypeTumorPred",
                                                 sample_info_column="orig.ident")

In [ ]:
edge_info_list <- c(lung_edge_info_list,liver_edge_info_list,colon_edge_info_list,prostate_edge_info_list)
combined_fgsea_dt <- data.table()
pathways <- load_pathways()
hallmark_pathways <- names(pathways)[grepl("HALLMARK_",names(pathways))] %>% gsub("HALLMARK_","",.)
num_hallmark <- sum(grepl("HALLMARK",names(pathways)))
p_values <- c()
for (cell_type in names(edge_info_list)) {
    edge_info <- edge_info_list[[cell_type]]
    for (normal_pc in names(edge_info)) {
        combined_pc <- paste("combined",edge_info[[normal_pc]]$combined_PCs_used,sep="_")
        combined_pc_label <- combined_pc %>% gsub("_","",.)
        normal_pc_label <- gsub("normal_","",normal_pc) %>% gsub("_","",.)

        dt <- edge_info[[normal_pc]]$fgsea_dt 
        c <- edge_info[[normal_pc]]$pca_cor_df %>% dplyr::filter(Var1 == normal_pc & Var2 == combined_pc) %>% pull(cor)
        combined_fgsea_dt <- rbind(combined_fgsea_dt,
                                   edge_info[[normal_pc]]$fgsea_dt %>% 
                                   mutate(cell_type=paste(paste(cell_type,
                                                          paste(normal_pc_label,
                                                                gsub("combined","",combined_pc_label),sep="-")),
                                         paste("C=",round(c,2)),sep="\n")))
        
        dt <- dt[grepl("HALLMARK",pathway),]
        enriched_pathways <- dt[padj < 0.1,pathway] %>% gsub("HALLMARK_","",.)
        not_enriched_pathways <- dt[padj >= 0.1,pathway] %>% gsub("HALLMARK_","",.)
        num_in_pc_in_mascaux <- length(intersect(enriched_pathways,mascaux_gene_sets))
        num_in_pc_not_in_mascaux <- length(intersect(enriched_pathways,setdiff(hallmark_pathways,mascaux_gene_sets)))
        num_not_in_pc_in_mascaux <- length(intersect(not_enriched_pathways,mascaux_gene_sets))
        num_not_in_pc_not_in_mascaux <- length(intersect(not_enriched_pathways,setdiff(hallmark_pathways,mascaux_gene_sets)))
        fisher_res <- fisher.test(matrix(c(num_in_pc_in_mascaux,num_in_pc_not_in_mascaux,
                                          num_not_in_pc_in_mascaux,num_not_in_pc_not_in_mascaux),
                                 nrow=2,ncol=2,byrow=T),alternative="g")
        print(cell_type)
        print(normal_pc)
        print(fisher_res$p.value)
        p_values <- c(p_values,fisher_res$p.value)
        flush.console()
    }
}

disp_fgsea_mat <- combined_fgsea_dt[,.(pathway,padj,NES,cell_type)] %>% mutate(NES=ifelse(padj < 0.05,NES,NA)) %>%
dplyr::select(-padj) %>% tidyr::pivot_wider(names_from="cell_type",values_from="NES") %>%
tibble::column_to_rownames("pathway") %>% as.matrix

## Figure 5B, S3D

In [ ]:
disp_fgsea_mat <- combined_fgsea_dt[,.(pathway,padj,NES,cell_type)] %>% mutate(NES=ifelse(padj < 0.05,NES,0)) %>%
dplyr::select(-padj) %>% tidyr::pivot_wider(names_from="cell_type",values_from="NES") %>%
tibble::column_to_rownames("pathway") %>% as.matrix
cols_to_plot <- c('Club PC1-PC1\nC= 0.95','AT1 PC1-PC1\nC= 0.96','AT2 PC5-PC1\nC= 0.26','Hep2 PC2-PC1\nC= -0.22',
                  'Hep3 PC2-PC2\nC= -0.07','TA 2 PC2-PC5\nC= 0.14','Enterocytes PC5-PC1\nC= -0.17',
                  'Stem PC1-PC3\nC= -0.81','TA 1 PC2-PC3\nC= 0.39','Enterocyte Progenitors PC1-PC3\nC= 0.58',
                  'Tuft PC1-PC3\nC= 0.9','Epi_Basal_hillock PC1-PC1\nC= -0.48','Epi_Luminal_1 PC5-PC4\nC= 0.23')
mascaux_mat <- disp_fgsea_mat[paste("HALLMARK",mascaux_gene_sets,sep="_"),cols_to_plot]
vec <- na.omit(as.vector(mascaux_mat))
colors <- colorRamp2(c(min(vec),0,max(vec)),c("#ffa500","white","#ff0000"))
values <- c(1.2,1.4,1.6,1.8,2)
p_heatmap <- Heatmap(mascaux_mat,cluster_rows=T,cluster_columns=F,
        name="Norm. Enrichment\nScore",
                     na_col="white", 
       rect_gp = gpar(col = "black", lwd = 2))
ggsave( "Fig5b.svg", grid.grabExpr(draw(p_heatmap)), width=10, height=7 )

p_heatmap_full <- Heatmap(disp_fgsea_mat[rowSums(disp_fgsea_mat == 0) < ncol(disp_fgsea_mat),],cluster_rows=T,cluster_columns=F,
        name="Norm. Enrichment\nScore",
                     na_col="white", 
       rect_gp = gpar(col = "black", lwd = 1), column_names_gp=gpar(fontsize=8),
                         row_names_gp=gpar(fontsize=8))
ggsave( "Fig-S3D.svg", grid.grabExpr(draw(p_heatmap_full)), width=10, height=8 )

## Mouse PDAC models

### GSE129455 (KPC mouse)


Hardly any acinar cells in this data!

In [ ]:
gene_exp_dt <- fread(file.path(base_path,"GSE129455_All_Viable_expression.csv.gz"))
mart <- useMart(biomart="ENSEMBL_MART_ENSEMBL",#"ENSEMBL_MART_FUNCGEN", 
                     host="ensembl.org", dataset="mmusculus_gene_ensembl")
gene_ids <- gene_exp_dt$V1
gene_name_dt <- getBM( attributes=c("external_gene_name","ensembl_gene_id"),#,"go_id"),
                 filters="ensembl_gene_id", values=gene_ids, mart=mart, verbose=F ) %>% data.table(.)
gene_exp_mat <- merge(gene_exp_dt,gene_name_dt,by.x="V1",by.y="ensembl_gene_id") %>% dplyr::select(-V1) %>%
tibble::column_to_rownames("external_gene_name") %>% as.matrix
gse129455_obj <- CreateSeuratObject( gene_exp_mat, project="GSE129455" )
meta_data_df <- gse129455_obj@meta.data %>% tibble::rownames_to_column("cell.name") %>%
mutate(sample_ID=gsub("..*?-","",cell.name)) %>% tibble::column_to_rownames("cell.name")
gse129455_obj <- AddMetaData(gse129455_obj,meta_data_df)
gse129455_obj[["RNA"]]@data <- gse129455_obj[["RNA"]]@counts

In [ ]:
gse129455_obj <- FindVariableFeatures(gse129455_obj) %>% ScaleData %>% RunPCA(.,npcs=20) %>%
FindNeighbors %>% RunUMAP(.,dims=1:20) %>% FindClusters(res=0.5)

marker_genes <- c("Apoe","Saa3","C1qc","Ear2","Lyz1","Cd79a","Ly6d", 
                                  "Ms4a1","S100a8","S100a9","G0s2","Dcn","Col1a1","Col3a1", 
                                  "Ccl5","Cd3g","Gzma","Nkg7","Clu","Tff1","Krt18","Krt8",
                                  "Krt19","Krt7","Epcam","Igfbp7","Plvap","Cd34","Ccr7", 
                                  "Ccl22","Ctrb1","Prss2","Try5","Rgs5","Acta2", "Cdkn2a",
                                  "S100a6","Igfbp4","Sparc","Vim","Spp1")

mat <- FetchData( gse129455_obj, vars=c("seurat_clusters",marker_genes )) %>%
group_by(seurat_clusters) %>% summarize_at(marker_genes,mean) %>% tibble::column_to_rownames("seurat_clusters") %>%
as.matrix %>% scale 
pheatmap( mat )

### GSE141017

In [ ]:
source("../src/process_pdac.R")

In [ ]:
gse141017_obj <- load_gse141017_data() %>% subset(idents="Acinar")
gse141017_obj$orig.ident <- paste("GSE141017",gse141017_obj$orig.ident,sep="-")
gse141017_obj <- NormalizeData( gse141017_obj )

In [ ]:
options(repr.plot.height=10,repr.plot.width=10)
DotPlot( gse141017_obj, features=c("tdTomato","Krt19","Sox9","Muc5ac","Gkn1","Gkn2",
                                   "Tlf1","Muc6","Pga5","Pgc","Try4","Cpa1","Cpa2","Cela2a",
                                   "Amy2a2","Reg3b","Krt19","Mmp7","Amy1") %>% unique) +
theme(axis.text.x=element_text(angle=90,hjust=1,vjust=0.5))

### GSE125588

In [ ]:
source("../src/process_pdac.R")
gse125588_obj_list <- load_gse125588_data()

### Tabula muris + Tabula muris senis

In [ ]:
muris_obj@meta.data$mouse.id %>% unique

In [ ]:
source("../src/process_pdac.R")
muris_obj <- load_muris_data()
muris_senis_obj <- load_muris_senis_data()

### Merging all mouse data

In [ ]:
obj_list <- c(SplitObject(muris_senis_obj,split.by="orig.ident"),
              SplitObject(muris_obj,split.by="orig.ident"),
              SplitObject(gse141017_obj,split.by="orig.ident"),
              gse125588_obj_list)
merged_obj <- NULL
idx <- 1
for (obj_name in names(obj_list)) {
    if (is.null(merged_obj)) {
        merged_obj <- obj_list[[obj_name]]
    } else {
        merged_obj <- merge( merged_obj, obj_list[[obj_name]] )
    }
    obj_list[[obj_name]] <- NormalizeData(obj_list[[obj_name]]) %>% FindVariableFeatures
}

anchors <- FindIntegrationAnchors( obj_list, k.filter=30 )
mouse_integrated_obj <- IntegrateData( anchors, k.weight=30 )
mouse_integrated_obj <- ScaleData(mouse_integrated_obj) %>% RunPCA(.,npcs=30) %>%
RunUMAP(.,dims=1:30)

In [ ]:
mouse_integrated_obj <- readRDS(file.path(base_path,"Mouse_Integrated_Obj.rds"))
df <- FetchData(mouse_integrated_obj,vars=c("Kras_status","is_edge","orig.ident")) %>%
group_by(orig.ident,Kras_status) %>% summarize(frac=mean(is_edge))

p_mouse <- ggdotplot( df, x="Kras_status",y="frac", fill="black", add="boxplot") +
theme_pubr(base_size=15) + ylab("Edge cell fraction") + labs(fill="Kras status")

wilcox.test( df %>% dplyr::filter(Kras_status == "Kras-mut") %>% pull(frac),
            df %>% dplyr::filter(Kras_status == "WT") %>% pull(frac) )
wilcox.test( df %>% dplyr::filter(Kras_status == "Kras-mut" & 
                                  orig.ident %in% c("GSE125588_early_KIC","GSE141017-PRT17")) %>% pull(frac),
            df %>% dplyr::filter(Kras_status == "WT") %>% pull(frac) )

In [ ]:
options(repr.plot.height=7,repr.plot.width=7)
p1 <- DimPlot( mouse_integrated_obj, group.by="orig.ident" ) + theme_pubr(base_size=13.33) + 
theme(legend.position="none") + ggtitle(NULL)
p2 <- DimPlot( mouse_integrated_obj, group.by="Kras_status", cols=c("black","gray")) + theme_pubr(base_size=13.33) +
ggtitle(NULL) + theme(legend.position="none")
mouse_integrated_obj$is_edge <- factor(mouse_integrated_obj$is_edge,levels=c(1,0)) 
p3 <- DimPlot( mouse_integrated_obj, group.by="is_edge" ) + theme_pubr(base_size=13.33) + ggtitle(NULL) +
theme(legend.position="none")

In [ ]:
mouse_integrated_obj <- SetIdent(mouse_integrated_obj,value="is_edge")
# wt_integrated_obj <- subset(mouse_integrated_obj,subset = Kras_status=="WT")
# kras_integrated_obj <- subset(mouse_integrated_obj,subset = Kras_status!="WT")
# wt_de_df <- FindMarkers(mouse_integrated_obj,ident.1="1",test.use="MAST",
#                         latent.vars=c("orig.ident"),logfc.threshold=0) %>% tibble::rownames_to_column("gene")
mouse_integrated_obj <- SetIdent(mouse_integrated_obj,value="Kras_status")
kras_wt_de_df <- FindMarkers(mouse_integrated_obj,
                            ident.1="Kras-mut", logfc.threshold=0.0, test.use="MAST",
                            latent.vars=c("orig.ident")) %>% tibble::rownames_to_column("gene")
merged_de_df <- merge(kras_wt_de_df,wt_de_df,by="gene",all.x=T,all.y=T,
                     suffixes=c("_Kras_vs_WT","_Edge"))
fwrite(merged_de_df,"Edge_Non_Edge_Kras_DE.tsv",sep="\t")
p_edge_kras_logFC <- ggplot( merged_de_df ) + geom_point(aes(x=avg_log2FC_Kras_vs_WT,y=avg_log2FC_Edge)) +
theme_pubr(base_size=13.33)

In [ ]:
lr_markers_df_list <- readRDS("all_outlier_markers.rds")
edge_acinar_markers_df <- lr_markers_df_list[["Acinar cell"]] %>% dplyr::filter(p_val_adj < 0.1)

edge_gene_set <- list("edge"=edge_acinar_markers_df %>% dplyr::filter(avg_logFC > 0) %>% pull(gene) %>%
                     get_orthologs(.,from_species="human",to_species="mouse") %>% pull(mouse_gene_name))    

combined_auc_df <- data.frame()
for (sample_name in unique(mouse_integrated_obj$orig.ident)) {
    subset_obj <- subset( mouse_integrated_obj, subset = orig.ident == sample_name)
    output <- compute_AUCell_scores(subset_obj,edge_gene_set,compute_thresholds=F)
    auc_df <- reshape2::melt(output$auc_mat) %>% dplyr::select(-`gene sets`) %>% mutate(sample=sample_name)
    combined_auc_df <- rbind(combined_auc_df,auc_df)
    print(sample_name)
    flush.console()
}

control_sd_df <- compute_shuffled_gene_set_AUCell_scores(mouse_integrated_obj, edge_gene_set)
merged_df <- merge( combined_auc_df,
    control_sd_df %>% group_by(sample) %>% summarize(m=quantile(mean,1.0)) %>% dplyr::rename(thresh=m),
                   by="sample" ) %>% mutate(is_edge=ifelse(value > thresh,1,0))

mouse_integrated_obj <- 
AddMetaData( mouse_integrated_obj,
            merged_df %>% dplyr::select(cells,is_edge,edge_AUC=value) %>% tibble::column_to_rownames("cells") )

mouse_integrated_obj <- AddMetaData( mouse_integrated_obj, mouse_integrated_obj@meta.data %>% 
                                    mutate(Kras_status=ifelse(!grepl("CTRL|normal",orig.ident) & !grepl("Muris",orig.ident),
                                                              "Kras-mut","WT")))

saveRDS(mouse_integrated_obj,file.path(base_path,"Mouse_Integrated_Obj.rds"))

### Mouse pancreatitis bulk RNA-seq data

In [ ]:
source("../src/process_pdac.R")
library(DESeq2)
library(tximport)
library(readxl)
library(TxDb.Mmusculus.UCSC.mm10.ensGene) 
txdb <- TxDb.Mmusculus.UCSC.mm10.ensGene

k <- keys(txdb, keytype = "GENEID")
df <- select(txdb, keys = k,  columns = "TXNAME", keytype = "GENEID") %>% dplyr::select(TXNAME,GENEID)
lr_markers_df_list <- readRDS("all_outlier_markers.rds")
edge_acinar_markers_df <- lr_markers_df_list[["Acinar cell"]] %>% dplyr::filter(p_val_adj < 0.1)

mouse_edge_gene_set <- list("edge"=edge_acinar_markers_df %>% dplyr::filter(avg_logFC > 0) %>% pull(gene) %>%
                     get_orthologs(.,from_species="human",to_species="mouse") %>% pull(mouse_gene_name)) 

source("../src/process_pdac.R")
gse106550_df <- process_GSE106550(mouse_edge_gene_set)
gse102675_df <- process_GSE102675(mouse_edge_gene_set)
gse143748_df <- process_GSE143749(mouse_edge_gene_set)
gse84659_df <- process_GSE84659(mouse_edge_gene_set)
gse132330_df <- process_GSE132330(mouse_edge_gene_set)
mouse_de_seq_res_df <- rbind(gse106550_df,gse102675_df,gse143748_df,gse84659_df,gse132330_df)
p_bulk_pancreatitis <- ggplot( mouse_de_seq_res_df %>% 
       mutate(accession=ifelse(accession %in% c("GSE84659","GSE132330"),
                               paste(comparison,accession,sep="_"),accession))) + 
geom_boxplot(aes(x=accession,y=log2FoldChange,fill=gene_type),
                                            outlier.shape=NA) + ylim(-7.5,7.5) + theme_pubr(base_size=13.33)+
theme(legend.position="none")

In [ ]:
wilcox.test( df %>% dplyr::filter(Kras_status == "WT") %>% pull(frac),
            df %>% dplyr::filter(Kras_status == "Kras-mut") %>% pull(frac) )

In [ ]:
wilcox.test(gse106550_df %>% dplyr::filter(gene_type=="Edge") %>% pull(log2FoldChange),
           gse106550_df %>% dplyr::filter(gene_type != "Edge") %>% pull(log2FoldChange),
           alternative="g")
wilcox.test(gse102675_df %>% dplyr::filter(gene_type=="Edge") %>% pull(log2FoldChange),
           gse102675_df %>% dplyr::filter(gene_type != "Edge") %>% pull(log2FoldChange),
           alternative="g")
wilcox.test(gse143748_df %>% dplyr::filter(gene_type=="Edge") %>% pull(log2FoldChange),
           gse143748_df %>% dplyr::filter(gene_type != "Edge") %>% pull(log2FoldChange),
           alternative="g")
wilcox.test(gse84659_df %>% dplyr::filter(comparison == "8h-Control" & gene_type=="Edge") %>% pull(log2FoldChange),
           gse84659_df %>% dplyr::filter(comparison == "8h-Control" & gene_type != "Edge") %>% pull(log2FoldChange),
           alternative="g")
wilcox.test(gse84659_df %>% dplyr::filter(comparison == "24h-Control" & gene_type=="Edge") %>% pull(log2FoldChange),
           gse84659_df %>% dplyr::filter(comparison == "24h-Control" & gene_type != "Edge") %>% pull(log2FoldChange),
           alternative="g")
wilcox.test(gse84659_df %>% dplyr::filter(comparison == "48h-Control" & gene_type=="Edge") %>% pull(log2FoldChange),
           gse84659_df %>% dplyr::filter(comparison == "48h-Control" & gene_type != "Edge") %>% pull(log2FoldChange),
           alternative="g")
wilcox.test(gse132330_df %>% dplyr::filter(comparison == "Injury_vs_Normal" & gene_type=="Edge") %>% pull(log2FoldChange),
           gse132330_df %>% dplyr::filter(comparison == "Injury_vs_Normal" & gene_type != "Edge") %>% pull(log2FoldChange),
           alternative="g")
wilcox.test(gse132330_df %>% dplyr::filter(comparison == "KrasG12D_vs_Normal" & gene_type=="Edge") %>% pull(log2FoldChange),
           gse132330_df %>% dplyr::filter(comparison == "KrasG12D_vs_Normal" & gene_type != "Edge") %>% pull(log2FoldChange),
           alternative="g")
wilcox.test(gse132330_df %>% dplyr::filter(comparison == "KrasG12DInjury_vs_Normal" & gene_type=="Edge") %>% pull(log2FoldChange),
           gse132330_df %>% dplyr::filter(comparison == "KrasG12DInjury_vs_Normal" & gene_type != "Edge") %>% pull(log2FoldChange),
           alternative="g")

In [ ]:
gse132330_df %>% dplyr::filter(gene_type == "Edge") %>%
dplyr::select(gene_name,comparison,log2FoldChange) %>% tidyr::pivot_wider(names_from="comparison",
                                                                         values_from="log2FoldChange",
                                                                         values_fn=mean) -> gse132330_wide_logfc_df
wilcox.test(gse132330_wide_logfc_df$KrasG12DInjury_vs_Normal,
           gse132330_wide_logfc_df$Injury_vs_Normal,alternative="g")
wilcox.test(gse132330_wide_logfc_df$KrasG12DInjury_vs_Normal,
           gse132330_wide_logfc_df$KrasG12D_vs_Normal,alternative="g")

### Figure 7

In [ ]:
options(repr.plot.width=12)
p_fig7 <- grid.arrange(
    arrangeGrob(p1,p2,#p3,
                p_mouse, 
                p_edge_kras_logFC + xlab("Log2-FC (Kras vs WT)") + ylab("Log2-FC (Edge vs Non-Edge)"),
                p_bulk_pancreatitis + ylab("Log2-FC"),
    layout_matrix=rbind(c(1,2,3),c(4,5,5)),
    ncol=3
))
# p_fig7 <- ggarrange(p1,p2,#p3,
#                     p_mouse,
#                     p_edge_kras_logFC,
#                     p_bulk_pancreatitis,
#          nrow=2,ncol=3)
ggsave("Fig7.svg",p_fig7,width=9,height=5)